In [41]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time, os
from fake_useragent import UserAgent
import datetime
from tqdm import tqdm
import pickle
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import unidecode

pd.set_option("display.max_rows", 300)

# Trialing Wiki comparisons with select pages
Notes:
- Tirana page
    - 1849 wikilinks -> 1400 unique wikilinks (using set()) -> 1398 unique wikilinks without "" and " ") 
        - compare_places(tirana, toronto) = 245
    - ... -> 1188 wikilinks after find("div", id="bodyContent")
        - compare_places(tirana, toronto) = 75

In [2]:
def get_wiki_page(url, wikilinks=False):
    '''Takes a URL as a string, collects and parses the HTML using Beautiful Soup and returns either the          parsed soup or a list of wikilinks as strings.

    Arguments: string (URL)
    Returns: list of strings OR parsed HTML as string
    '''

    response = requests.get(url)
    print(response.status_code)
    page = response.text
    soup = BeautifulSoup(page)

    if wikilinks==True:
        page_wikilinks = list(set([wikilink.text for wikilink in soup.find("div", id="bodyContent").find_all("a", href=re.compile("(\/wiki\/.+)")) if wikilink.text != "" and wikilink.text != " "]))
        return page_wikilinks
    else:
        return soup

def compare_places(place1_links, place2_links):
    '''Takes two lists of wikilinks (strings) and returns the number of wikiilnks shared betweeen them.

    Arguments: list of strings, list of strings
    Returns: int
    '''

    counter = 0
    for link in place1_links:
        if link in place2_links:
            counter += 1

    return counter

In [3]:
tirana = get_wiki_page("https://en.wikipedia.org/wiki/Tirana", wikilinks=True)

200


In [4]:
len(tirana)

1188

In [5]:
toronto = get_wiki_page("https://en.wikipedia.org/wiki/Toronto", wikilinks=True)

200


In [6]:
compare_places(tirana, toronto)

75

In [7]:
montreal = get_wiki_page("https://en.wikipedia.org/wiki/Montreal", wikilinks=True)

200


In [8]:
quebec_city = get_wiki_page("https://en.wikipedia.org/wiki/Quebec_City", wikilinks=True)

200


In [9]:
compare_places(toronto, montreal)

274

In [10]:
compare_places(montreal, quebec_city)

145

In [13]:
montreal[:10]

['Hydro-Québec Building',
 'Akwesasne',
 'Cabot Square',
 'Pages with non-numeric formatnum arguments',
 'Quebec City–Windsor Corridor',
 'Saint-Jean-Baptiste²',
 'Articles with French-language sources (fr)',
 'Ojibwe language',
 'Quebecor',
 'Montreal City Council']

In [28]:
response = requests.get("https://en.wikipedia.org/wiki/Budapest")
print(response.status_code)
page = response.text
soup = BeautifulSoup(page)

200


In [29]:
souper = BeautifulSoup('''<ul><li><b><a href="/wiki/Amsterdam" title="Amsterdam">Amsterdam</a></b>,&nbsp;Netherlands<sup>1</sup></li>
<li><b><a href="/wiki/Andorra_la_Vella" title="Andorra la Vella">Andorra la Vella</a></b>,&nbsp;Andorra</li>
<li><b><a href="/wiki/Ankara" title="Ankara">Ankara</a></b>,&nbsp;Turkey<sup>3</sup></li>
<li><b><a href="/wiki/Athens" title="Athens">Athens</a></b>,&nbsp;Greece</li>
<li><b><a href="/wiki/Baku" title="Baku">Baku</a></b>,&nbsp;Azerbaijan<sup>3</sup></li>
<li><b><a href="/wiki/Belgrade" title="Belgrade">Belgrade</a></b>,&nbsp;Serbia</li>
<li><b><a href="/wiki/Berlin" title="Berlin">Berlin</a></b>,&nbsp;Germany</li>
<li><b><a href="/wiki/Bern" title="Bern">Bern</a></b>,&nbsp;Switzerland</li>
<li><b><a href="/wiki/Bratislava" title="Bratislava">Bratislava</a></b>,&nbsp;Slovakia</li>
<li><b><a href="/wiki/Brussels" title="Brussels">Brussels</a></b>,&nbsp;Belgium<sup>2</sup></li>
<li><b><a href="/wiki/Bucharest" title="Bucharest">Bucharest</a></b>,&nbsp;Romania</li>
<li><b><a class="mw-selflink selflink">Budapest</a></b>,&nbsp;Hungary</li>
<li><b><a href="/wiki/Chi%C8%99in%C4%83u" title="Chișinău">Chișinău</a></b>,&nbsp;Moldova</li>
<li><b><a href="/wiki/Copenhagen" title="Copenhagen">Copenhagen</a></b>,&nbsp;Denmark</li>
<li><b><a href="/wiki/Dublin" title="Dublin">Dublin</a></b>,&nbsp;Ireland</li>
<li><b><a href="/wiki/Helsinki" title="Helsinki">Helsinki</a></b>,&nbsp;Finland</li>
<li><b><a href="/wiki/Kyiv" title="Kyiv">Kyiv</a></b>,&nbsp;Ukraine</li>
<li><b><a href="/wiki/Lisbon" title="Lisbon">Lisbon</a></b>,&nbsp;Portugal</li>
<li><b><a href="/wiki/Ljubljana" title="Ljubljana">Ljubljana</a></b>,&nbsp;Slovenia</li>
<li><b><a href="/wiki/London" title="London">London</a></b>,&nbsp;United Kingdom</li>
<li><b><a href="/wiki/Luxembourg_City" title="Luxembourg City">Luxembourg</a></b>,&nbsp;Luxembourg</li>
<li><b><a href="/wiki/Madrid" title="Madrid">Madrid</a></b>,&nbsp;Spain</li>
<li><b><a href="/wiki/Minsk" title="Minsk">Minsk</a></b>,&nbsp;Belarus</li>
<li><b><a href="/wiki/Monaco" title="Monaco">Monaco</a></b><sup>5</sup></li>
<li><b><a href="/wiki/Moscow" title="Moscow">Moscow</a></b>,&nbsp;Russia<sup>3</sup></li>
<li><b><a href="/wiki/Nicosia" title="Nicosia">Nicosia</a></b>,&nbsp;Cyprus<sup>4</sup></li>
<li><b><a href="/wiki/Nur-Sultan" title="Nur-Sultan">Nur-Sultan</a></b>,&nbsp;Kazakhstan<sup>3</sup></li>
<li><b><a href="/wiki/Oslo" title="Oslo">Oslo</a></b>,&nbsp;Norway</li>
<li><b><a href="/wiki/Paris" title="Paris">Paris</a></b>,&nbsp;France</li>
<li><b><a href="/wiki/Podgorica" title="Podgorica">Podgorica</a></b>,&nbsp;Montenegro</li>
<li><b><a href="/wiki/Prague" title="Prague">Prague</a></b>,&nbsp;Czech Republic</li>
<li><b><a href="/wiki/Reykjav%C3%ADk" title="Reykjavík">Reykjavík</a></b>,&nbsp;Iceland</li>
<li><b><a href="/wiki/Riga" title="Riga">Riga</a></b>,&nbsp;Latvia</li>
<li><b><a href="/wiki/Rome" title="Rome">Rome</a></b>,&nbsp;Italy</li>
<li><b><a href="/wiki/City_of_San_Marino" title="City of San Marino">San Marino</a></b>,&nbsp;San Marino</li>
<li><b><a href="/wiki/Sarajevo" title="Sarajevo">Sarajevo</a></b>,&nbsp;Bosnia and Herzegovina</li>
<li><b><a href="/wiki/Skopje" title="Skopje">Skopje</a></b>,&nbsp;North Macedonia</li>
<li><b><a href="/wiki/Sofia" title="Sofia">Sofia</a></b>,&nbsp;Bulgaria</li>
<li><b><a href="/wiki/Stockholm" title="Stockholm">Stockholm</a></b>,&nbsp;Sweden</li>
<li><b><a href="/wiki/Tallinn" title="Tallinn">Tallinn</a></b>,&nbsp;Estonia</li>
<li><b><a href="/wiki/Tbilisi" title="Tbilisi">Tbilisi</a></b>,&nbsp;Georgia<sup>3</sup></li>
<li><b><a href="/wiki/Tirana" title="Tirana">Tirana</a></b>,&nbsp;Albania</li>
<li><b><a href="/wiki/Vaduz" title="Vaduz">Vaduz</a></b>,&nbsp;Liechtenstein</li>
<li><b><a href="/wiki/Valletta" title="Valletta">Valletta</a></b>,&nbsp;Malta</li>
<li><b><a href="/wiki/Vatican_City" title="Vatican City">Vatican City</a></b><sup>5</sup></li>
<li><b><a href="/wiki/Vienna" title="Vienna">Vienna</a></b>,&nbsp;Austria</li>
<li><b><a href="/wiki/Vilnius" title="Vilnius">Vilnius</a></b>,&nbsp;Lithuania</li>
<li><b><a href="/wiki/Warsaw" title="Warsaw">Warsaw</a></b>,&nbsp;Poland</li>
<li><b><a href="/wiki/Yerevan" title="Yerevan">Yerevan</a></b>,&nbsp;Armenia<sup>4</sup></li>
<li><b><a href="/wiki/Zagreb" title="Zagreb">Zagreb</a></b>,&nbsp;Croatia</li></ul>''')

In [45]:
european_cap_links = []
caps = []
for a in souper.find_all("a", href=True):
    if a != '<a class="mw-selflink selflink">Budapest</a>':
        sublink = a["href"]
        european_cap_links.append(f'https://en.wikipedia.org{sublink}')
        caps.append(a.text)

In [47]:
european_cap_list = []
for link in european_cap_links:
    european_cap_list.append(get_wiki_page(link, wikilinks=True))

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [3]:
def city_compare(names, soups, print_all=True):
    
    main_dict = dict(zip(names, soups))
    best_matches = {}

    for city, links in main_dict.items():
        if print_all==True:
            print(f'\n{city}\n***********\n')
        best_score = 0
        closest_city = ''
        for other_city in main_dict.keys():
            if other_city != city:
                score = compare_places(main_dict[city], main_dict[other_city])
                if print_all==True:
                    print(f'{city} x {other_city}: {score}')
                if score > best_score:
                    best_score = score
                    closest_city = other_city
        if print_all==True:
            print(f'* CLOSEST MATCH, {city.upper()}: {closest_city.upper()} *')
        best_matches[city] = (closest_city, best_score)

    return best_matches

In [69]:
city_compare(caps, european_cap_list)

Valletta: 310
Riga x Vatican City: 298
Riga x Vienna: 405
Riga x Vilnius: 367
Riga x Warsaw: 293
Riga x Yerevan: 255
Riga x Zagreb: 283
* CLOSEST MATCH, RIGA: TALLINN *

Rome
***********

Rome x Amsterdam: 216
Rome x Andorra la Vella: 89
Rome x Ankara: 143
Rome x Athens: 214
Rome x Baku: 172
Rome x Belgrade: 146
Rome x Berlin: 236
Rome x Bern: 119
Rome x Bratislava: 148
Rome x Brussels: 207
Rome x Bucharest: 136
Rome x Chișinău: 116
Rome x Copenhagen: 179
Rome x Dublin: 184
Rome x Helsinki: 201
Rome x Kyiv: 192
Rome x Lisbon: 168
Rome x Ljubljana: 157
Rome x London: 113
Rome x Luxembourg: 174
Rome x Madrid: 205
Rome x Minsk: 99
Rome x Monaco: 174
Rome x Moscow: 144
Rome x Nicosia: 130
Rome x Nur-Sultan: 119
Rome x Oslo: 185
Rome x Paris: 241
Rome x Podgorica: 106
Rome x Prague: 159
Rome x Reykjavík: 125
Rome x Riga: 177
Rome x San Marino: 88
Rome x Sarajevo: 154
Rome x Skopje: 125
Rome x Sofia: 160
Rome x Stockholm: 226
Rome x Tallinn: 184
Rome x Tbilisi: 110
Rome x Tirana: 138
Rome x 

{'Amsterdam': ('Paris', 413),
 'Andorra la Vella': ('Madrid', 249),
 'Ankara': ('Baku', 371),
 'Athens': ('Stockholm', 468),
 'Baku': ('Madrid', 400),
 'Belgrade': ('Vienna', 350),
 'Berlin': ('Stockholm', 477),
 'Bern': ('Vienna', 263),
 'Bratislava': ('Vienna', 360),
 'Brussels': ('Stockholm', 472),
 'Bucharest': ('Paris', 307),
 'Chișinău': ('Bucharest', 273),
 'Copenhagen': ('Stockholm', 524),
 'Dublin': ('Madrid', 468),
 'Helsinki': ('Stockholm', 453),
 'Kyiv': ('Madrid', 419),
 'Lisbon': ('Madrid', 375),
 'Ljubljana': ('Zagreb', 301),
 'London': ('Paris', 175),
 'Luxembourg': ('Brussels', 454),
 'Madrid': ('Stockholm', 486),
 'Minsk': ('Moscow', 281),
 'Monaco': ('Vatican City', 419),
 'Moscow': ('Paris', 341),
 'Nicosia': ('Baku', 371),
 'Nur-Sultan': ('Baku', 378),
 'Oslo': ('Stockholm', 497),
 'Paris': ('Berlin', 427),
 'Podgorica': ('Belgrade', 261),
 'Prague': ('Berlin', 352),
 'Reykjavík': ('Stockholm', 381),
 'Riga': ('Tallinn', 533),
 'Rome': ('Vatican City', 247),
 'San 

In [70]:
souper = BeautifulSoup('''<a href="/wiki/Ainu_cuisine" title="Ainu cuisine">Ainu</a>
<a href="/wiki/Albanian_cuisine" title="Albanian cuisine">Albanian</a>
<a href="/wiki/Argentinian_cuisine" class="mw-redirect" title="Argentinian cuisine">Argentine</a>
<a href="/wiki/Telugu_cuisine" title="Telugu cuisine">Andhra</a>
<a href="/wiki/American_cuisine" title="American cuisine">American</a>
<a href="/wiki/Anglo-Indian_cuisine" title="Anglo-Indian cuisine">Anglo-Indian</a>
<a href="/wiki/Arab_cuisine" title="Arab cuisine">Arab</a>
<a href="/wiki/Armenian_cuisine" title="Armenian cuisine">Armenian</a>
<a href="/wiki/Assyrian_cuisine" title="Assyrian cuisine">Assyrian</a>
<a href="/wiki/Awadhi_cuisine" title="Awadhi cuisine">Awadhi</a>
<a href="/wiki/Azerbaijani_cuisine" title="Azerbaijani cuisine">Azerbaijani</a>
<a href="/wiki/Balochi_cuisine" title="Balochi cuisine">Balochi</a>
<a href="/wiki/Belarusian_cuisine" title="Belarusian cuisine">Belarusian</a>
<a href="/wiki/Bangladeshi_cuisine" title="Bangladeshi cuisine">Bangladeshi</a>
<a href="/wiki/Bengali_cuisine" title="Bengali cuisine">Bengali</a>
<a href="/wiki/Berber_cuisine" title="Berber cuisine">Berber</a>
<a href="/wiki/Brazilian_cuisine" title="Brazilian cuisine">Brazilian</a>
<a href="/wiki/British_cuisine" title="British cuisine">British</a>
<a href="/wiki/Buddhist_cuisine" title="Buddhist cuisine">Buddhist</a>
<a href="/wiki/Bulgarian_cuisine" title="Bulgarian cuisine">Bulgarian</a>
<a href="/wiki/Cajun_cuisine" title="Cajun cuisine">Cajun</a>
<a href="/wiki/Cantonese_cuisine" title="Cantonese cuisine">Cantonese</a>
<a href="/wiki/Caribbean_cuisine" title="Caribbean cuisine">Caribbean</a>
<a href="/wiki/Chechen_cuisine" title="Chechen cuisine">Chechen</a>
<a href="/wiki/Chinese_cuisine" title="Chinese cuisine">Chinese cuisine</a>
<a href="/wiki/Chinese_Islamic_cuisine" title="Chinese Islamic cuisine">Chinese Islamic</a>
<a href="/wiki/Circassian_cuisine" title="Circassian cuisine">Circassian</a>
<a href="/wiki/Crimean_Tatar_cuisine" title="Crimean Tatar cuisine">Crimean Tatar</a>
<a href="/wiki/Cypriot_cuisine" title="Cypriot cuisine">Cypriot</a>
<a href="/wiki/Czech_cuisine" title="Czech cuisine">Czech</a>
<a href="/wiki/Danish_cuisine" title="Danish cuisine">Danish</a>
<a href="/wiki/Egyptian_cuisine" title="Egyptian cuisine">Egyptian</a>
<a href="/wiki/English_cuisine" title="English cuisine">English</a>
<a href="/wiki/Ethiopian_cuisine" title="Ethiopian cuisine">Ethiopian</a>
<a href="/wiki/Eritrean_cuisine" title="Eritrean cuisine">Eritrean</a>
<a href="/wiki/Estonian_cuisine" title="Estonian cuisine">Estonian</a>
<a href="/wiki/French_cuisine" title="French cuisine">French</a>
<a href="/wiki/Filipino_cuisine" title="Filipino cuisine">Filipino</a>
<a href="/wiki/Georgian_cuisine" title="Georgian cuisine">Georgian</a>
<a href="/wiki/German_cuisine" title="German cuisine">German</a>
<a href="/wiki/Goan_cuisine" title="Goan cuisine">Goan</a>
<a href="/wiki/Goan_Catholic_cuisine" title="Goan Catholic cuisine">Goan Catholic</a>
<a href="/wiki/Greek_cuisine" title="Greek cuisine">Greek</a>
<a href="/wiki/Gujarati_cuisine" title="Gujarati cuisine">Gujarati</a>
<a href="/wiki/Hyderabadi_cuisine" title="Hyderabadi cuisine">Hyderabad</a>
<a href="/wiki/Indian_cuisine" title="Indian cuisine">Indian cuisine</a>
<a href="/wiki/Indian_Chinese_cuisine" title="Indian Chinese cuisine">Indian Chinese</a>
<a href="/wiki/Indian_Singaporean_cuisine" title="Indian Singaporean cuisine">Indian Singaporean cuisine</a>
<a href="/wiki/Indonesian_cuisine" title="Indonesian cuisine">Indonesian</a>
<a href="/wiki/Inuit_diet" class="mw-redirect" title="Inuit diet">Inuit</a>
<a href="/wiki/Irish_cuisine" title="Irish cuisine">Irish</a>
<a href="/wiki/Italian-American_cuisine" title="Italian-American cuisine">Italian-American</a>
<a href="/wiki/Italian_cuisine" title="Italian cuisine">Italian cuisine</a>
<a href="/wiki/Jamaican_cuisine" title="Jamaican cuisine">Jamaican</a>
<a href="/wiki/Japanese_cuisine" title="Japanese cuisine">Japanese</a>
<a href="/wiki/Jewish_cuisine" title="Jewish cuisine">Jewish</a>
<a href="/wiki/Cuisine_of_Karnataka" title="Cuisine of Karnataka">Karnataka</a>
<a href="/wiki/Kazakh_cuisine" title="Kazakh cuisine">Kazakh</a>
<a href="/wiki/Kerala_cuisine" class="mw-redirect" title="Kerala cuisine">Keralite</a>
<a href="/wiki/Korean_cuisine" title="Korean cuisine">Korean</a>
<a href="/wiki/Kurdish_cuisine" title="Kurdish cuisine">Kurdish</a>
<a href="/wiki/Laotian_cuisine" class="mw-redirect" title="Laotian cuisine">Laotian</a>
<a href="/wiki/Lebanese_cuisine" title="Lebanese cuisine">Lebanese</a>
<a href="/wiki/Latvian_cuisine" title="Latvian cuisine">Latvian</a>
<a href="/wiki/Lithuanian_cuisine" title="Lithuanian cuisine">Lithuanian</a>
<a href="/wiki/Louisiana_Creole_cuisine" title="Louisiana Creole cuisine">Louisiana Creole</a>
<a href="/wiki/Maharashtrian_cuisine" title="Maharashtrian cuisine">Maharashtrian</a>
<a href="/wiki/Mangalorean_cuisine" title="Mangalorean cuisine">Mangalorean</a>
<a href="/wiki/Malay_cuisine" title="Malay cuisine">Malay</a>
<a href="/wiki/Malaysian_Chinese_cuisine" title="Malaysian Chinese cuisine">Malaysian Chinese cuisine</a>
<a href="/wiki/Malaysian_Indian_cuisine" title="Malaysian Indian cuisine">Malaysian Indian cuisine</a>
<a href="/wiki/Mediterranean_cuisine" title="Mediterranean cuisine">Mediterranean cuisine</a>
<a href="/wiki/Mennonite_cuisine" title="Mennonite cuisine">Mennonite</a>
<a href="/wiki/Mexican_cuisine" title="Mexican cuisine">Mexican</a>
<a href="/wiki/Mordovian_cuisine" title="Mordovian cuisine">Mordovian</a>
<a href="/wiki/Mughlai_cuisine" title="Mughlai cuisine">Mughal</a>
<a href="/wiki/Native_American_cuisine" class="mw-redirect" title="Native American cuisine">Native American</a>
<a href="/wiki/Nepalese_cuisine" title="Nepalese cuisine">Nepalese</a>
<a href="/wiki/New_Mexican_cuisine" title="New Mexican cuisine">New Mexican</a>
<a href="/wiki/Odia_cuisine" class="mw-redirect" title="Odia cuisine">Odia</a>
<a href="/wiki/Parsi_cuisine" title="Parsi cuisine">Parsi</a>
<a href="/wiki/Pashtun_cuisine" title="Pashtun cuisine">Pashtun</a>
<a href="/wiki/Polish_cuisine" title="Polish cuisine">Polish</a>
<a href="/wiki/Cuisine_of_the_Pennsylvania_Dutch" title="Cuisine of the Pennsylvania Dutch">Pennsylvania Dutch</a>
<a href="/wiki/Pakistani_cuisine" title="Pakistani cuisine">Pakistani</a>
<a href="/wiki/Peranakan_cuisine" title="Peranakan cuisine">Peranakan</a>
<a href="/wiki/Persian_cuisine" class="mw-redirect" title="Persian cuisine">Persian</a>
<a href="/wiki/Peruvian_cuisine" title="Peruvian cuisine">Peruvian</a>
<a href="/wiki/Portuguese_cuisine" title="Portuguese cuisine">Portuguese</a>
<a href="/wiki/Punjabi_cuisine" title="Punjabi cuisine">Punjabi</a>
<a href="/wiki/Cuisine_of_Quebec" title="Cuisine of Quebec">Québécois</a>
<a href="/wiki/Rajasthani_cuisine" title="Rajasthani cuisine">Rajasthani</a>
<a href="/wiki/Romanian_cuisine" title="Romanian cuisine">Romanian</a>
<a href="/wiki/Russian_cuisine" title="Russian cuisine">Russian</a>
<a href="/wiki/Sami_cuisine" title="Sami cuisine">Sami</a>
<a href="/wiki/Serbian_cuisine" title="Serbian cuisine">Serbian</a>
<a href="/wiki/Sindhi_cuisine" title="Sindhi cuisine">Sindhi</a>
<a href="/wiki/Slovak_cuisine" title="Slovak cuisine">Slovak</a>
<a href="/wiki/Slovenian_cuisine" title="Slovenian cuisine">Slovenian</a>
<a href="/wiki/Somali_cuisine" title="Somali cuisine">Somali</a>
<a href="/wiki/South_Indian_cuisine" title="South Indian cuisine">South Indian</a>
<a href="/wiki/Soviet_cuisine" title="Soviet cuisine">Soviet</a>
<a href="/wiki/Spanish_cuisine" title="Spanish cuisine">Spanish</a>
<a href="/wiki/Sri_Lankan_cuisine" title="Sri Lankan cuisine">Sri Lankan</a>
<a href="/wiki/Taiwanese_cuisine" title="Taiwanese cuisine">Taiwanese</a>
<a href="/wiki/Tatar_cuisine" title="Tatar cuisine">Tatar</a>
<a href="/wiki/Texan_cuisine" title="Texan cuisine">Texan</a>
<a href="/wiki/Thai_cuisine" title="Thai cuisine">Thai</a>
<a href="/wiki/Turkish_cuisine" title="Turkish cuisine">Turkish</a>
<a href="/wiki/Tamil_cuisine" title="Tamil cuisine">Tamil</a>
<a href="/wiki/Udupi_cuisine" title="Udupi cuisine">Udupi</a>
<a href="/wiki/Ukrainian_cuisine" title="Ukrainian cuisine">Ukrainian</a>
<a href="/wiki/Vietnamese_cuisine" title="Vietnamese cuisine">Vietnamese</a>
<a href="/wiki/Yamal_cuisine" title="Yamal cuisine">Yamal</a>
<a href="/wiki/Zambian_cuisine" title="Zambian cuisine">Zambian</a>
<a href="/wiki/Zanzibari_cuisine" title="Zanzibari cuisine">Zanzibari</a>''')

cuisine_links = []
cuisines = []
for a in souper.find_all("a", href=True):
    if a != '<a class="mw-selflink selflink">Budapest</a>':
        sublink = a["href"]
        cuisine_links.append(f'https://en.wikipedia.org{sublink}')
        cuisines.append(a.text)

In [72]:
city_compare(cuisines, cuisine_links)

x Spanish: 44
Turkish x Sri Lankan: 44
Turkish x Taiwanese: 45
Turkish x Tatar: 45
Turkish x Texan: 45
Turkish x Thai: 45
Turkish x Tamil: 45
Turkish x Udupi: 44
Turkish x Ukrainian: 44
Turkish x Vietnamese: 44
Turkish x Yamal: 44
Turkish x Zambian: 44
Turkish x Zanzibari: 44
* CLOSEST MATCH, TURKISH: ANDHRA *

Tamil
***********

Tamil x Ainu: 40
Tamil x Albanian: 41
Tamil x Argentine: 40
Tamil x Andhra: 42
Tamil x American: 41
Tamil x Anglo-Indian: 41
Tamil x Arab: 40
Tamil x Armenian: 41
Tamil x Assyrian: 40
Tamil x Awadhi: 40
Tamil x Azerbaijani: 40
Tamil x Balochi: 41
Tamil x Belarusian: 41
Tamil x Bangladeshi: 41
Tamil x Bengali: 41
Tamil x Berber: 40
Tamil x Brazilian: 41
Tamil x British: 40
Tamil x Buddhist: 40
Tamil x Bulgarian: 41
Tamil x Cajun: 40
Tamil x Cantonese: 40
Tamil x Caribbean: 40
Tamil x Chechen: 40
Tamil x Chinese cuisine: 40
Tamil x Chinese Islamic: 42
Tamil x Circassian: 40
Tamil x Crimean Tatar: 42
Tamil x Cypriot: 40
Tamil x Czech: 40
Tamil x Danish: 40
Tamil 

{'Ainu': ('Albanian', 42),
 'Albanian': ('Anglo-Indian', 45),
 'Argentine': ('Ainu', 49),
 'Andhra': ('Tamil', 44),
 'American': ('Armenian', 46),
 'Anglo-Indian': ('Italian-American', 50),
 'Arab': ('Albanian', 42),
 'Armenian': ('American', 46),
 'Assyrian': ('Ainu', 45),
 'Awadhi': ('Ainu', 44),
 'Azerbaijani': ('Albanian', 47),
 'Balochi': ('Belarusian', 45),
 'Belarusian': ('Balochi', 48),
 'Bangladeshi': ('Balochi', 49),
 'Bengali': ('Balochi', 45),
 'Berber': ('Albanian', 43),
 'Brazilian': ('Balochi', 46),
 'British': ('Balochi', 45),
 'Buddhist': ('Balochi', 46),
 'Bulgarian': ('Balochi', 47),
 'Cajun': ('Azerbaijani', 42),
 'Cantonese': ('Cajun', 47),
 'Caribbean': ('Québécois', 47),
 'Chechen': ('Cajun', 45),
 'Chinese cuisine': ('Cajun', 45),
 'Chinese Islamic': ('Italian-American', 52),
 'Circassian': ('Cajun', 48),
 'Crimean Tatar': ('Chinese Islamic', 50),
 'Cypriot': ('Malaysian Chinese cuisine', 45),
 'Czech': ('Azerbaijani', 42),
 'Danish': ('Pennsylvania Dutch', 44),

In [73]:
vietnamese_cuisine = get_wiki_page("https://en.wikipedia.org/wiki/Vietnamese_cuisine", wikilinks=True)

200


In [113]:
vietnamese_cuisine[:10]

['bánh khoái',
 'Japanese',
 'Sarawakian',
 'Commons category link from Wikidata',
 'crackers',
 'Phở',
 'Guava',
 'Government',
 'Pot',
 'Macedonian']

In [104]:
test = vietnamese_cuisine

for item in vietnamese_cuisine:
    if re.match("(Wikipedia articles)", item):
        print(re.match("(Wikipedia articles)", item))
        print(item)

<re.Match object; span=(0, 18), match='Wikipedia articles'>
Wikipedia articles with SUDOC identifiers
<re.Match object; span=(0, 18), match='Wikipedia articles'>
Wikipedia articles needing clarification from September 2009
<re.Match object; span=(0, 18), match='Wikipedia articles'>
Wikipedia articles with BNF identifiers
<re.Match object; span=(0, 18), match='Wikipedia articles'>
Wikipedia articles with LCCN identifiers
<re.Match object; span=(0, 18), match='Wikipedia articles'>
Wikipedia articles with NDL identifiers


In [102]:
len(vietnamese_cuisine)

1208

In [95]:
len(test)

1208

In [ ]:
Wikipedia articles

In [ ]:
"(Region topic)", "(Wikipedia articles)", "(Commons category link from Wikidata)", "(Articles with)", "(.+-\d+-\d+-.+)", "(( |^)eat( |$|\b))", "(All articles with)"

In [106]:
vietnamese_cuisine[:10]

['bánh khoái',
 'Japanese',
 'Sarawakian',
 'Commons category link from Wikidata',
 'crackers',
 'Phở',
 'Guava',
 'Government',
 'Pot',
 'Macedonian']

In [107]:
argentine_cuisine = get_wiki_page("https://en.wikipedia.org/wiki/Argentine_cuisine", wikilinks=True)

200


In [111]:
compare_places(argentine_cuisine, vietnamese_cuisine)

428

In [109]:
uruguayan_cuisine = get_wiki_page("https://en.wikipedia.org/wiki/Uruguayan_cuisine", wikilinks=True)

200


In [112]:
compare_places(argentine_cuisine, uruguayan_cuisine)

478

## Taking title from "a" tags instead of tag as they are more consistent
- "carpinchos" on Uruguay cuisine page links to "Capibara" page

In [99]:
def get_cuisine_links(url, country, bottom_nav_check=False, text_only=False, raw_text=False):

    import requests
    import re
    from bs4 import BeautifulSoup
    from tqdm import tqdm
    import pickle
    import unidecode

    response = requests.get(url)
    print(response.status_code)
    page = response.text
    soup = BeautifulSoup(page)

    # Each page should include itself in its links, as others that link to it will share something with it
    self_link = url.split("https://en.wikipedia.org/wiki/")[1]
    title = soup.find(id="firstHeading").text
    bottom_nav_links = np.NaN
    if bottom_nav_check==True:
        for i, a in enumerate(soup.find_all("a", class_="mw-selflink selflink")):
            if a.text == f'Cuisine of {country}' or a.text == title:
                bottom_nav_links = []
                print(a.text)
                try:
                    for _ in soup.find(id="External_links").find_next("td", class_="navbox-list").find_all("a"):
                        bottom_nav_links.append((_["href"].lstrip('/wiki/'), _.text))
                except (AttributeError, KeyError) as error:
                    continue
                bottom_nav_links = list(set(bottom_nav_links))

    tags_to_extract = ['table', 'script', 'meta', 'style']
    titles_to_extract = ['Jump up', 'Enlarge', re.compile("(Wikipedia:)"), "Help:Category", re.compile("(Category:Wiki.*)"), re.compile("(Category:Comm.*)"), re.compile("(Category:Articles.*)"), re.compile("(Category:All )"), 'ISBN (identifier)', re.compile("(Special:)"), re.compile("(Category:CS1)")]
    texts_to_extract = ['edit', re.compile("(Jump to)"), 'cuisine']
    classes_to_extract = ['reference']
    ids_to_extract = ['mw-hidden-catlinks']

    for t in tags_to_extract:
        [s.extract() for s in soup(t)]
    for t in ids_to_extract:
        [s.extract() for s in soup(id=t)]
    for t in titles_to_extract:
        [s.extract() for s in soup("a", title=t)]
    for t in texts_to_extract:
        [s.extract() for s in soup("a", text=t)]
    for t in classes_to_extract:
        [s.extract() for s in soup("a", class_=t)]

    main_body = soup.find('div', id='bodyContent')
    if text_only==False:
        page_wikilinks = [(country, country), (self_link, title)] + list(set([(wikilink["href"].lstrip('/wiki/'), wikilink.text) for wikilink in main_body.find_all("a", href=re.compile("(^\/wiki\/.+)")) if wikilink.text != "" and wikilink.text != " "]))
    else:
        page_wikilinks = [(country, country), (self_link, title)] + list(set([unidecode.unidecode(wikilink.text) for wikilink in main_body.find_all("a", href=re.compile("(^\/wiki\/.+)")) if wikilink.text != "" and wikilink.text != " "]))
    
    if bottom_nav_check==True:
        return page_wikilinks, bottom_nav_links
    else:
        return page_wikilinks

In [248]:
len(set(list(page_wikilinks)))

340

In [342]:
title

'Armenian cuisine'

## Getting Africa templates

In [6]:
response = requests.get("https://en.wikipedia.org/wiki/Category:Africa_country_and_territory_topics_templates")
page = response.text
soup = BeautifulSoup(page)


africa_templates = [(a.text, f'https://en.wikipedia.org{a["href"]}') for a in soup.find("div", id="mw-pages").find("div", class_="mw-content-ltr").find_all("a")]
# Looking over what was pulled
for i, place in enumerate(africa_templates):
    print(i, place[0])
africa_templates = africa_templates[1:4] + africa_templates[5:17] + africa_templates[18:42] + africa_templates[43:45] + africa_templates[46:51] + africa_templates[52:54] + africa_templates[55:]

0 Template:Africa topic templates
1 Template:Algeria topics
2 Template:Angola topics
3 Template:Benin topics
4 Template:Biafra topics
5 Template:Botswana topics
6 Template:Burkina Faso topics
7 Template:Burundi topics
8 Template:Cameroon topics
9 Template:Cape Verde topics
10 Template:Central African Republic topics
11 Template:Chad topics
12 Template:Comoros topics
13 Template:Democratic Republic of the Congo topics
14 Template:Republic of the Congo topics
15 Template:Djibouti topics
16 Template:Egypt topics
17 Template:Ancient Egypt topics
18 Template:Equatorial Guinea topics
19 Template:Eritrea topics
20 Template:Eswatini topics
21 Template:Ethiopia topics
22 Template:Gabon topics
23 Template:Gambia topics
24 Template:Ghana topics
25 Template:Guinea topics
26 Template:Guinea-Bissau topics
27 Template:Ivory Coast topics
28 Template:Kenya topics
29 Template:Lesotho topics
30 Template:Liberia topics
31 Template:Libya topics
32 Template:Madagascar topics
33 Template:Malawi topics
34 Tem

In [7]:
response = requests.get("https://en.wikipedia.org/wiki/Category:Middle_East_country_and_territory_topics_templates")
page = response.text
soup = BeautifulSoup(page)


middle_east_templates = [(a.text, f'https://en.wikipedia.org{a["href"]}') for a in soup.find("div", id="mw-pages").find("div", class_="mw-content-ltr").find_all("a")]
# Looking over what was pulled
for i, place in enumerate(middle_east_templates):
    print(i, place[0])
middle_east_templates = middle_east_templates[:3] + middle_east_templates[4:11] + middle_east_templates[12:13] + middle_east_templates[15:19] + middle_east_templates[20:]
for i, place in enumerate(middle_east_templates):
    print(i, place[0])


0 Template:Bahrain topics
1 Template:Djibouti topics
2 Template:Egypt topics
3 Template:Ancient Egypt topics
4 Template:Iran topics
5 Template:Iraq topics
6 Template:Iraqi Kurdistan topics
7 Template:Israel topics
8 Template:Jordan topics
9 Template:Kuwait topics
10 Template:Lebanon topics
11 Template:Mandatory Palestine topics
12 Template:Oman topics
13 Template:Ottoman Empire topics
14 Template:Palestine (historic region) topics
15 Template:Palestine topics
16 Template:Qatar topics
17 Template:Saudi Arabia topics
18 Template:Syria topics
19 Template:Syrian Civil War map
20 Template:Turkey topics
21 Template:United Arab Emirates topics
22 Template:Yemen topics
0 Template:Bahrain topics
1 Template:Djibouti topics
2 Template:Egypt topics
3 Template:Iran topics
4 Template:Iraq topics
5 Template:Iraqi Kurdistan topics
6 Template:Israel topics
7 Template:Jordan topics
8 Template:Kuwait topics
9 Template:Lebanon topics
10 Template:Oman topics
11 Template:Palestine topics
12 Template:Qatar t

In [8]:
response = requests.get("https://en.wikipedia.org/wiki/Category:Asia_country_and_territory_topics_templates")
page = response.text
soup = BeautifulSoup(page)


asia_templates = [(a.text, f'https://en.wikipedia.org{a["href"]}') for a in soup.find("div", class_="mw-category-generated").find_all("a")]
# Looking over what was pulled
# for i, place in enumerate(asia_templates):
#     print(i, place[0])
asia_templates = [('Template:Philippines topics', 'https://en.wikipedia.org/wiki/Template:Philippines_topics')] + asia_templates[5:11] + asia_templates[12:16] + asia_templates[17:36] + asia_templates[37:39] + asia_templates[40:47]
for i, place in enumerate(asia_templates):
    print(i, place[0])



0 Template:Philippines topics
1 Template:Abkhazia topics
2 Template:Afghanistan topics
3 Template:Armenia topics
4 Template:Artsakh topics
5 Template:Azerbaijan topics
6 Template:Bangladesh topics
7 Template:Bhutan topics
8 Template:Brunei topics
9 Template:Cambodia topics
10 Template:China topics
11 Template:East Timor topics
12 Template:Georgia (country) topics
13 Template:Hong Kong topics
14 Template:India topics
15 Template:South India topics
16 Template:Indonesia topics
17 Template:Japan topics
18 Template:Kazakhstan topics
19 Template:Korea topics
20 Template:Kyrgyzstan topics
21 Template:Laos topics
22 Template:Macau topics
23 Template:Malaysia topics
24 Template:Maldives topics
25 Template:Mongolia topics
26 Template:Myanmar topics
27 Template:Nepal topics
28 Template:Pakistan topics
29 Template:Russia topics
30 Template:Singapore topics
31 Template:South Ossetia topics
32 Template:Sri Lanka topics
33 Template:Taiwan topics
34 Template:Tajikistan topics
35 Template:Thailand top

In [9]:
response = requests.get("https://en.wikipedia.org/wiki/Category:Europe_country_and_territory_topics_templates")
page = response.text
soup = BeautifulSoup(page)


europe_templates = [(a.text, f'https://en.wikipedia.org{a["href"]}') for a in soup.find("div", class_="mw-category-generated").find_all("a")]
# Looking over what was pulled
# for i, place in enumerate(asia_templates):
#     print(i, place[0])
europe_templates = europe_templates[6:23] + europe_templates[24:27] + europe_templates[28:50] + europe_templates[51:54] + europe_templates[55:80]
for i, place in enumerate(europe_templates):
    print(i, place[0])

0 Template:Albania topics
1 Template:Alderney topics
2 Template:Andorra topics
3 Template:Armenia topics
4 Template:Asturias topics
5 Template:Austria topics
6 Template:Azerbaijan topics
7 Template:Belarus topics
8 Template:Belgium topics
9 Template:Bulgaria topics
10 Template:Cantabria topics
11 Template:Catalonia topics
12 Template:Crimea topics
13 Template:Croatia topics
14 Template:Cyprus topics
15 Template:Northern Cyprus topics
16 Template:Czech Republic topics
17 Template:Denmark topics
18 Template:England topics
19 Template:Estonia topics
20 Template:Faroe Islands topics
21 Template:Finland topics
22 Template:Flanders topics
23 Template:France topics
24 Template:Galicia topics
25 Template:Georgia (country) topics
26 Template:Germany topics
27 Template:Gibraltar topics
28 Template:Greece topics
29 Template:Greenland topics
30 Template:Guernsey topics
31 Template:Hungary topics
32 Template:Iceland topics
33 Template:Ireland topics
34 Template:Italy topics
35 Template:Jersey topic

In [10]:
response = requests.get("https://en.wikipedia.org/wiki/Category:North_America_country_and_territory_topics_templates")
page = response.text
soup = BeautifulSoup(page)


north_america_templates = [(a.text, f'https://en.wikipedia.org{a["href"]}') for a in soup.find("div", class_="mw-category-generated").find_all("a")]
# Looking over what was pulled
# for i, place in enumerate(asia_templates):
#     print(i, place[0])
north_america_templates = north_america_templates[4:]
for i, place in enumerate(north_america_templates):
    print(i, place[0])

0 Template:Bermuda topics
1 Template:Canada topics
2 Template:Greenland topics
3 Template:Mexico topics
4 Template:Ontario topics
5 Template:Puerto Rico topics
6 Template:Quebec topics
7 Template:United States topics


In [11]:
response = requests.get("https://en.wikipedia.org/wiki/Category:Oceania_country_and_territory_topics_templates")
page = response.text
soup = BeautifulSoup(page)


oceania_templates = [(a.text, f'https://en.wikipedia.org{a["href"]}') for a in soup.find("div", class_="mw-category-generated").find_all("a")]
# Looking over what was pulled
# for i, place in enumerate(asia_templates):
#     print(i, place[0])
oceania_templates = oceania_templates[1:]
for i, place in enumerate(oceania_templates):
    print(i, place[0])

0 Template:Australia topics
1 Template:Cook Islands topics
2 Template:Federated States of Micronesia topics
3 Template:Fiji topics
4 Template:French Polynesia topics
5 Template:Kiribati topics
6 Template:Marshall Islands topics
7 Template:Nauru topics
8 Template:New Caledonia topics
9 Template:New Zealand topics
10 Template:Niue topics
11 Template:Palau topics
12 Template:Papua New Guinea topics
13 Template:Pitcairn topics
14 Template:Samoa topics
15 Template:Solomon Islands topics
16 Template:Tonga topics
17 Template:Tuvalu topics
18 Template:Vanuatu topics
19 Template:Wallis and Futuna topics


In [12]:
response = requests.get("https://en.wikipedia.org/wiki/Category:South_America_country_and_territory_topics_templates")
page = response.text
soup = BeautifulSoup(page)


south_america_templates = [(a.text, f'https://en.wikipedia.org{a["href"]}') for a in soup.find("div", class_="mw-category-generated").find_all("a")]
# Looking over what was pulled
# for i, place in enumerate(asia_templates):
#     print(i, place[0])
south_america_templates = south_america_templates[3:]
for i, place in enumerate(south_america_templates):
    print(i, place[0])

0 Template:Argentina topics
1 Template:Bolivia topics
2 Template:Brazil topics
3 Template:Chile topics
4 Template:Colombia topics
5 Template:Ecuador topics
6 Template:Falkland Islands
7 Template:Guyana topics
8 Template:Paraguay topics
9 Template:Peru topics
10 Template:Suriname topics
11 Template:Uruguay topics
12 Template:Venezuela topics


In [13]:
response = requests.get("https://en.wikipedia.org/wiki/Category:Central_America_country_and_territory_topics_templates")
page = response.text
soup = BeautifulSoup(page)


central_america_templates = [(a.text, f'https://en.wikipedia.org{a["href"]}') for a in soup.find("div", class_="mw-category-generated").find_all("a")]
# Looking over what was pulled
# for i, place in enumerate(asia_templates):
#     print(i, place[0])
central_america_templates = central_america_templates[3:4] + central_america_templates[5:]
for i, place in enumerate(central_america_templates):
    print(i, place[0])

0 Template:Belize topics
1 Template:Costa Rica topics
2 Template:El Salvador topics
3 Template:Guatemala topics
4 Template:Honduras topics
5 Template:Nicaragua topics
6 Template:Panama topics


In [14]:
response = requests.get("https://en.wikipedia.org/wiki/Category:Caribbean_country_and_territory_topics_templates")
page = response.text
soup = BeautifulSoup(page)


caribbean_templates = [(a.text, f'https://en.wikipedia.org{a["href"]}') for a in soup.find("div", class_="mw-category-generated").find_all("a")]
# Looking over what was pulled
# for i, place in enumerate(asia_templates):
#     print(i, place[0])
caribbean_templates = caribbean_templates[3:]
for i, place in enumerate(caribbean_templates):
    print(i, place[0])

0 Template:Antigua and Barbuda topics
1 Template:Aruba topics
2 Template:The Bahamas topics
3 Template:Barbados topics
4 Template:British Virgin Islands topics
5 Template:Cayman Islands topics
6 Template:Cuba topics
7 Template:Curaçao topics
8 Template:Dominica topics
9 Template:Dominican Republic topics
10 Template:Grenada topics
11 Template:Guadeloupe topics
12 Template:Haiti topics
13 Template:Jamaica topics
14 Template:Martinique topics
15 Template:Montserrat topics
16 Template:Puerto Rico topics
17 Template:Saint Kitts and Nevis topics
18 Template:Saint Lucia topics
19 Template:Saint Vincent and the Grenadines topics
20 Template:Trinidad and Tobago topics
21 Template:Turks and Caicos topics


In [83]:
 def get_cuisine_dict(list_of_templates):
 
    cuisine_dict = {}
    for group in list_of_templates:
        if group == africa_templates:
            continent = "Africa"
        elif group == middle_east_templates:
            continent = "Middle East"
        elif group == asia_templates:
            continent = "Asia"
        elif group == europe_templates:
            continent = "Europe"
        elif group == north_america_templates:
            continent = "North America"
        elif group == oceania_templates:
            continent = "Oceania"
        elif group == south_america_templates:
            continent = "South America"
        elif group == central_america_templates:
            continent = "Central America"
        elif group == caribbean_templates:
            continent = "Caribbean"
        for template in group:
            country = " ".join(template[0].split(":")[1].split(" ")[:-1])
            response = requests.get(f'{template[1]}')
            page = response.text
            soup = BeautifulSoup(page)
            country_links, bottom_links = np.NaN, np.NaN
            for a in soup.find("div", class_="navbox").find_all("a"):
                if a.text == "Cuisine":
                    cuisine_link = f'https://en.wikipedia.org{a["href"]}'
                    country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            if country == "Angola":
                cuisine_link = 'https://en.wikipedia.org/wiki/Angolan_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Burundi":
                cuisine_link = 'https://en.wikipedia.org/wiki/Burundian_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Chad":
                cuisine_link = 'https://en.wikipedia.org/wiki/Chadian_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Comoros" or country == "Madagascar":
                cuisine_link = 'https://en.wikipedia.org/wiki/Malagasy_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Comoros" or country == "Madagascar":
                cuisine_link = 'https://en.wikipedia.org/wiki/Cuisine_of_Eswatini'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Djiboutii":
                cuisine_link = 'https://en.wikipedia.org/wiki/Djiboutian_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Gabon":
                cuisine_link = 'https://en.wikipedia.org/wiki/Gabonese_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Gambia":
                cuisine_link = 'https://en.wikipedia.org/wiki/Gambian_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Ivory Coast":
                cuisine_link = 'https://en.wikipedia.org/wiki/Ivorian_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Kenya":
                country_links = kenya_links
                bottom_links = np.NaN
            elif country == "Lesotho":
                cuisine_link = 'https://en.wikipedia.org/wiki/Cuisine_of_Lesotho'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Libya":
                cuisine_link = 'https://en.wikipedia.org/wiki/Libyan_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Malawi":
                cuisine_link = 'https://en.wikipedia.org/wiki/Malawian_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Mauritania":
                cuisine_link = 'https://en.wikipedia.org/wiki/Mauritanian_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Mauritania":
                cuisine_link = 'https://en.wikipedia.org/wiki/Mauritanian_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Mauritius":
                cuisine_link = 'https://en.wikipedia.org/wiki/Cuisine_of_Mauritius'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Mozambique":
                cuisine_link = 'https://en.wikipedia.org/wiki/Cuisine_of_Mozambique'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Niger":
                cuisine_link = 'https://en.wikipedia.org/wiki/Cuisine_of_Niger'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Réunion":
                cuisine_link = 'https://en.wikipedia.org/wiki/Gastronomy_of_Reuni%C3%B3n'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Réunion":
                cuisine_link = 'https://en.wikipedia.org/wiki/Gastronomy_of_Reuni%C3%B3n'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "São Tomé and Príncipe":
                cuisine_link = 'https://en.wikipedia.org/wiki/Cuisine_of_S%C3%A3o_Tom%C3%A9_and_Pr%C3%ADncipe'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Seychelles":
                cuisine_link = 'https://en.wikipedia.org/wiki/Seychellois_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "South Sudan":
                cuisine_link = 'https://en.wikipedia.org/wiki/South_Sudanese_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Sudan":
                cuisine_link = 'https://en.wikipedia.org/wiki/Sudanese_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Togo":
                cuisine_link = 'https://en.wikipedia.org/wiki/Togolese_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Zambia":
                cuisine_link = 'https://en.wikipedia.org/wiki/Zambian_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Zimbabwe":
                country_links = zimbabwe_links
                bottom_links = np.NaN
            elif country == "Iraqi Kurdistan":
                cuisine_link = 'https://en.wikipedia.org/wiki/Kurdish_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "South India":
                cuisine_link = 'https://en.wikipedia.org/wiki/South_Indian_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Korea":
                cuisine_link = 'https://en.wikipedia.org/wiki/Korean_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Crimea":
                cuisine_link = 'https://en.wikipedia.org/wiki/Crimean_Tatar_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Jersey":
                country_links = jersey_links
                bottom_links = np.NaN
            elif country == "Liechtenstein":
                cuisine_link = 'https://en.wikipedia.org/wiki/Liechtenstein_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Northern Ireland":
                cuisine_link = 'https://en.wikipedia.org/wiki/Northern_Irish_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Poland":
                cuisine_link = 'https://en.wikipedia.org/wiki/Polish_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Wales":
                cuisine_link = 'https://en.wikipedia.org/wiki/Welsh_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Bermuda":
                cuisine_link = 'https://en.wikipedia.org/wiki/Bermudian_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Bermuda":
                cuisine_link = 'https://en.wikipedia.org/wiki/Bermudian_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Cook Islands":
                cuisine_link = 'https://en.wikipedia.org/wiki/%27Ota_%27ika' #Specific dish, added because there isn't much here
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
                country_links = country_links + cook_islands_links
            elif country == "Federated States of Micronesia":
                cuisine_link = 'https://en.wikipedia.org/wiki/Cuisine_of_the_Mariana_Islands' #This may need to be changed
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Fiji":
                cuisine_link = 'https://en.wikipedia.org/wiki/Fijian_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "French Polynesia":
                cuisine_link1 = 'https://en.wikipedia.org/wiki/%27Ota_%27ika' #Specific dish, added because there isn't much here
                country_links1, bottom_links = get_cuisine_links(cuisine_link1, country, bottom_nav_check=True)
                cuisine_link2 = 'https://en.wikipedia.org/wiki/Po%27e' #Specific dish, added because there isn't much here
                country_links2, bottom_links = get_cuisine_links(cuisine_link2, country, bottom_nav_check=True)
                country_links = country_links1 + country_links2
            elif country == "Nauru":
                cuisine_link = 'https://en.wikipedia.org/wiki/Cuisine_of_Nauru'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True) 
            elif country == "Papua New Guinea":
                cuisine_link = 'https://en.wikipedia.org/wiki/Papua_New_Guinean_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)   
            elif country == "Samoa":
                cuisine_link = 'https://en.wikipedia.org/wiki/%27Ota_%27ika' #Specific dish, added because there isn't much here
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Tonga":
                cuisine_link = 'https://en.wikipedia.org/wiki/%27Ota_%27ika' #Specific dish, added because there isn't much here (also Tonga included here and I've already refered to it)
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
                country_links = country_links + tonga_links
            elif country == "Tuvalu":
                cuisine_link = 'https://en.wikipedia.org/wiki/Cuisine_of_Tuvalu'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "The Bahamas":
                cuisine_link = 'https://en.wikipedia.org/wiki/Bahamian_cuisine'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
            elif country == "Saint Kitts and Nevis":
                country_links = st_kitts_links
                bottom_links = np.NaN
            elif country == "Martinique":
                country_links = martinique_links
                bottom_links = np.NaN
            elif country == "Turks and Caicos":
                cuisine_link = 'https://en.wikipedia.org/wiki/Cuisine_of_the_Turks_and_Caicos_Islands'
                country_links, bottom_links = get_cuisine_links(cuisine_link, country, bottom_nav_check=True)
                
            cuisine_dict[country] = continent, country_links, bottom_links
    
    return cuisine_dict

In [84]:
country_cuisine_dict = get_cuisine_dict([africa_templates, middle_east_templates, asia_templates, europe_templates, north_america_templates, oceania_templates, south_america_templates, central_america_templates, caribbean_templates])

200
Algerian cuisine
200
200
200
200
200
200
200
200
200
200
200
200
200
Egyptian cuisine
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
Moroccan cuisine
Moroccan cuisine
200
200
200
200
200
200
200
200
200
200
200
200
South African cuisine
200
200
200
Culture of Tanzania
200
200
Cuisine of Tunisia
200
200
200
Zanzibari cuisine
200
200
Egyptian cuisine
200
Iranian cuisine
200
200
200
200
200
200
200
200
200
200
200
200
Turkish cuisine
200
200
200
Filipino cuisine
200
Georgian cuisine
200
200
Armenian cuisine
200
Azerbaijani cuisine
200
200
200
Bruneian cuisine
200
200
Chinese cuisine
200
200
Georgian cuisine
200
Hong Kong cuisine
Hong Kong cuisine
Hong Kong cuisine
200
Indian cuisine
Cuisine of India
200
200
Indonesian cuisine
Indonesian cuisine
200
200
200
Korean cuisine
200
200
Lao cuisine
200
Macanese cuisine
Macanese cuisine
200
Malaysian cuisine
Malaysian cuisine
200
200
200
Burmese cuisine
200
200
Pakistani cuisine
200
200
Singaporean cuisine
Singapor

In [85]:
tester = pd.DataFrame.from_dict(country_cuisine_dict, orient='index', columns=["Continent", "Cuisine links", "Bottom nav links"])
tester

,Continent,Cuisine links,Bottom nav links
Algeria,Africa,"[(Algeria, Algeria), (Algerian_cuisine, Algeri...","[(Chakhchoukha, Chakhchoukha), (Tajine, Tajine..."
Angola,Africa,"[(Angola, Angola), (Angolan_cuisine, Angolan c...",NaN
Benin,Africa,"[(Benin, Benin), (Benin_cuisine, Benin cuisine...",NaN
Botswana,Africa,"[(Botswana, Botswana), (Botswana_cuisine, Bots...",NaN
Burkina Faso,Africa,"[(Burkina Faso, Burkina Faso), (Burkinab%C3%A9...",NaN
Burundi,Africa,"[(Burundi, Burundi), (Burundian_cuisine, Burun...",NaN
Cameroon,Africa,"[(Cameroon, Cameroon), (Cuisine_of_Cameroon, C...",NaN
Cape Verde,Africa,"[(Cape Verde, Cape Verde), (Cape_Verdean_cuisi...",NaN
Central African Republic,Africa,"[(Central African Republic, Central African Re...",NaN
Chad,Africa,"[(Chad, Chad), (Chadian_cuisine, Chadian cuisi...",NaN


In [54]:
zimbabwe_raw = BeautifulSoup('''<div class="thumbinner" style="width:222px;"><a href="/wiki/File:Goat_Offal.JPG" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/d/d8/Goat_Offal.JPG/220px-Goat_Offal.JPG" decoding="async" width="220" height="165" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/d/d8/Goat_Offal.JPG/330px-Goat_Offal.JPG 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/d/d8/Goat_Offal.JPG/440px-Goat_Offal.JPG 2x" data-file-width="4352" data-file-height="3264"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Goat_Offal.JPG" class="internal" title="Enlarge"></a></div>A meal of sadza <i>(right)</i>, greens, and goat <a href="/wiki/Offal" title="Offal">offal</a>. The goat's small intestines are wrapped around small pieces of large intestines before cooking.</div></div><p>Like in many African countries, the majority of Zimbabweans depend on a few staple foods. "Mealie meal", also known as <a href="/wiki/Cornmeal" title="Cornmeal">cornmeal</a>, is used to prepare <i><a href="/wiki/Sadza" class="mw-redirect" title="Sadza">sadza</a></i> or <i>isitshwala</i>, as well as <a href="/wiki/Porridge" title="Porridge">porridge</a> known as <i>bota</i> or <i>ilambazi</i>. Sadza is made by mixing the cornmeal with water to produce a thick paste/porridge. After the paste has been cooking for several minutes, more cornmeal is added to thicken the paste.
</p><p><a href="/wiki/Graduation" title="Graduation">Graduations</a>, <a href="/wiki/Wedding" title="Wedding">weddings</a>, and any other family gatherings will usually be celebrated with the killing of a goat or cow, which will be barbecued or <a href="/wiki/Roasting" title="Roasting">roasted</a> by the family.
</p><p>This is usually eaten as lunch or dinner, usually with sides such as <a href="/wiki/Gravy" title="Gravy">gravy</a>, <a href="/wiki/Vegetable" title="Vegetable">vegetables</a> (<a href="/wiki/Spinach" title="Spinach">spinach</a>, <a href="/wiki/Chomolia" title="Chomolia">chomolia</a>, or <a href="/wiki/Spring_greens" title="Spring greens">spring greens</a>/<a href="/wiki/Collard_greens" class="mw-redirect" title="Collard greens">collard greens</a>), <a href="/wiki/Bean" title="Bean">beans</a>, and <a href="/wiki/Meat" title="Meat">meat</a> (stewed, grilled, roasted, or sundried). Sadza is also commonly eaten with curdled milk (<a href="/wiki/Sour_milk" class="mw-redirect" title="Sour milk">sour milk</a>), commonly known as "lacto" (<i>mukaka wakakora</i>), or dried <a href="/wiki/Tanganyika_sardine" class="mw-redirect" title="Tanganyika sardine">Tanganyika sardine</a>, known locally as <i>kapenta</i> or <i>matemba</i>. Bota is a thinner porridge, cooked without the additional cornmeal and usually flavoured with <a href="/wiki/Peanut_butter" title="Peanut butter">peanut butter</a>, milk, butter, or <a href="/wiki/Jam" class="mw-redirect" title="Jam">jam</a>.<sup id="cite_ref-263" class="reference"><a href="#cite_note-263">[263]</a></sup> Bota is usually eaten for breakfast.
</p><div class="thumb tright"><div class="thumbinner" style="width:222px;"><a href="/wiki/File:Boerewors_raw.jpg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/c/c6/Boerewors_raw.jpg/220px-Boerewors_raw.jpg" decoding="async" width="220" height="165" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/c/c6/Boerewors_raw.jpg/330px-Boerewors_raw.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/c/c6/Boerewors_raw.jpg/440px-Boerewors_raw.jpg 2x" data-file-width="450" data-file-height="338"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Boerewors_raw.jpg" class="internal" title="Enlarge"></a></div>Raw <a href="/wiki/Boerewors" title="Boerewors">boerewors</a></div></div></div><p>Even though the <a href="/wiki/Afrikaner" class="mw-redirect" title="Afrikaner">Afrikaners</a> are a small group (10%) within the white minority group, Afrikaner recipes are popular. <a href="/wiki/Biltong" title="Biltong">Biltong</a>, a type of <a href="/wiki/Jerky_(food)" class="mw-redirect" title="Jerky (food)">jerky</a>, is a popular snack, prepared by hanging bits of spiced raw meat to dry in the shade.<sup id="cite_ref-264" class="reference"><a href="#cite_note-264">[264]</a></sup> <a href="/wiki/Boerewors" title="Boerewors">Boerewors</a> is served with sadza. It is a long sausage, often well-spiced, composed of beef rather than pork, and barbecued.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (May 2016)">citation needed</span></a></i>]</sup> As Zimbabwe was a British colony, some people there have adopted some colonial-era English eating habits. For example, most people will have <a href="/wiki/Porridge" title="Porridge">porridge</a> in the morning, as well as 10 o'clock tea (midday tea). They will have lunch, often leftovers from the night before, freshly cooked <i>sadza</i>, or sandwiches (which is more common in the cities). After lunch, there is usually 4 o'clock tea (afternoon tea), which is served before dinner. It is not uncommon for <a href="/wiki/Tea" title="Tea">tea</a> to be had after dinner.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (May 2016)">citation needed</span></a></i>]</sup>
</p><p>Rice, pasta, and <a href="/wiki/Potato_production_in_Zimbabwe" title="Potato production in Zimbabwe">potato-based foods</a> (<a href="/wiki/French_fries" title="French fries">french fries</a> and <a href="/wiki/Mashed_potato" title="Mashed potato">mashed potato</a>) also make up part of Zimbabwean cuisine. A local favourite is rice cooked with peanut butter, which is taken with thick gravy, mixed vegetables and meat.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (May 2016)">citation needed</span></a></i>]</sup> A potpourri of peanuts known as <i>nzungu</i>, boiled and sundried maize, <a href="/wiki/Black-eyed_pea" title="Black-eyed pea">black-eyed peas</a> known as <i>nyemba</i>, and <a href="/wiki/Bambara_groundnut" class="mw-redirect" title="Bambara groundnut">bambara groundnuts</a> known as <i>nyimo</i> makes a traditional dish called <i>mutakura</i>. Mutakura can also be the above ingredients cooked individually. One can also find local snacks, such as <i>maputi</i> (roasted/popped maize kernels similar to <a href="/wiki/Popcorn" title="Popcorn">popcorn</a>), roasted and salted <a href="/wiki/Peanut" title="Peanut">peanuts</a>, <a href="/wiki/Sugar_cane" class="mw-redirect" title="Sugar cane">sugar cane</a>, <a href="/wiki/Sweet_potato" title="Sweet potato">sweet potato</a>, pumpkin, and indigenous fruits, such as <a href="/wiki/Horned_melon" class="mw-redirect" title="Horned melon">horned melon</a>, <i>gaka</i>, <i><a href="/wiki/Adansonia" title="Adansonia">adansonia</a></i>, <i>mawuyu</i>, <i><a href="/wiki/Uapaca_kirkiana" title="Uapaca kirkiana">uapaca kirkiana</a></i>, <i>mazhanje</i> (sugar plum), and many others.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (May 2016)">citation needed</span></a></i>]</sup>
</p>''')

In [56]:
soup = zimbabwe_raw

tags_to_extract = ['table', 'script', 'meta', 'style']
titles_to_extract = ['Jump up', 'Enlarge', re.compile("(Wikipedia:)"), "Help:Category", re.compile("(Category:Wiki.*)"), re.compile("(Category:Comm.*)"), re.compile("(Category:Articles.*)"), re.compile("(Category:All )"), 'ISBN_(identifier)', re.compile("(Special:)"), re.compile("(Category:CS1)")]
texts_to_extract = ['edit', re.compile("(Jump to)")]
classes_to_extract = ['reference']
ids_to_extract = ['mw-hidden-catlinks']

for t in tags_to_extract:
    [s.extract() for s in soup(t)]
for t in ids_to_extract:
    [s.extract() for s in soup(id=t)]
for t in titles_to_extract:
    [s.extract() for s in soup("a", title=t)]
for t in texts_to_extract:
    [s.extract() for s in soup("a", text=t)]
for t in classes_to_extract:
    [s.extract() for s in soup("a", class_=t)]

zimbabwe_links = [("Zimbabwe#Cuisine", "Zimbabwean cuisine")] + list(set([(wikilink["href"].lstrip('/wiki/'), wikilink.text) for wikilink in soup.find_all("a", href=re.compile("(^\/wiki\/.+)")) if wikilink.text != "" and wikilink.text != " "]))
zimbabwe_links

[('Zimbabwe#Cuisine', 'Zimbabwean cuisine'),
 ('Horned_melon', 'horned melon'),
 ('French_fries', 'french fries'),
 ('Sweet_potato', 'sweet potato'),
 ('Meat', 'meat'),
 ('Biltong', 'Biltong'),
 ('Wedding', 'weddings'),
 ('Spring_greens', 'spring greens'),
 ('Peanut_butter', 'peanut butter'),
 ('Bambara_groundnut', 'bambara groundnuts'),
 ('Jerky_(food)', 'jerky'),
 ('Cornmeal', 'cornmeal'),
 ('Boerewors', 'Boerewors'),
 ('Graduation', 'Graduations'),
 ('Spinach', 'spinach'),
 ('Sour_milk', 'sour milk'),
 ('Adansonia', 'adansonia'),
 ('Potato_production_in_Zimbabwe', 'potato-based foods'),
 ('Offal', 'offal'),
 ('Boerewors', 'boerewors'),
 ('Jam', 'jam'),
 ('Collard_greens', 'collard greens'),
 ('Tea', 'tea'),
 ('Mashed_potato', 'mashed potato'),
 ('Popcorn', 'popcorn'),
 ('Uapaca_kirkiana', 'uapaca kirkiana'),
 ('Peanut', 'peanuts'),
 ('Porridge', 'porridge'),
 ('Black-eyed_pea', 'black-eyed peas'),
 ('Bean', 'beans'),
 ('Tanganyika_sardine', 'Tanganyika sardine'),
 ('Sadza', 'sadza')

In [58]:
kenyan_cuisine_raw = BeautifulSoup('''<div class="thumb tright"><div class="thumbinner" style="width:222px;"><a href="/wiki/File:Ugali_with_beef_and_sauce.JPG" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/6/65/Ugali_with_beef_and_sauce.JPG/220px-Ugali_with_beef_and_sauce.JPG" decoding="async" width="220" height="165" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/6/65/Ugali_with_beef_and_sauce.JPG/330px-Ugali_with_beef_and_sauce.JPG 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/6/65/Ugali_with_beef_and_sauce.JPG/440px-Ugali_with_beef_and_sauce.JPG 2x" data-file-width="2592" data-file-height="1944"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Ugali_with_beef_and_sauce.JPG" class="internal" title="Enlarge"></a></div><a href="/wiki/Ugali" title="Ugali">Ugali</a> is a mainstay of the cuisine throughout the <a href="/wiki/African_Great_Lakes" title="African Great Lakes">African Great Lakes</a> region. Served here with beef and sauce.</div></div></div><p>There is no singular dish that represents all of Kenya's wide cuisine. Different communities have their own native foods. Staples are maize and other cereals depending on the region, including <a href="/wiki/Millet" title="Millet">millet</a> and <a href="/wiki/Sorghum" title="Sorghum">sorghum</a> eaten with various meats and vegetables. The foods that are universally eaten in Kenya are <a href="/wiki/Ugali" title="Ugali">ugali</a>, <a href="/wiki/Sukuma_wiki" title="Sukuma wiki">sukuma wiki</a>, and nyama choma. Kenya's coastal cuisine is unique and highly regarded throughout the country.
</p><p>Sukuma wiki, a <a href="/wiki/Swahili_language" title="Swahili language">Swahili</a> phrase which literally means "to push the week", is a simple dish made with greens similar to <a href="/wiki/Kale" title="Kale">kale</a> or <a href="/wiki/Collard_greens" class="mw-redirect" title="Collard greens">collards</a> that can also be made with cassava leaves, sweet potato leaves, or pumpkin leaves. Its Swahili name comes from the fact that it is typically eaten to "get through the week" or "stretch the week". Nyama choma is grilled meat, which can be considered as an unofficial national dish  – usually goat or sheep - while kuku choma is grilled chicken. It is usually cooked over an open fire. It is usually eaten with <a href="/wiki/Ugali" title="Ugali">ugali</a> and <a href="/wiki/Kachumbari" title="Kachumbari">kachumbari</a>, <a href="/w/index.php?title=Maharagwe&amp;action=edit&amp;redlink=1" class="new" title="Maharagwe (page does not exist)">maharagwe</a> (bean stew) and <a href="/wiki/Mchicha" class="mw-redirect" title="Mchicha">mchicha</a> (shredded spinach).	
</p><p>Among the <a href="/wiki/Luhya_people" title="Luhya people">Luhya</a> residing in the western region of the country, <i>ingokho</i> (<a href="/wiki/Chicken_(food)" class="mw-redirect" title="Chicken (food)">chicken</a>) and ugali is a favorite meal. Other than these, they also eat tsisaka, miroo, managu and other dishes. In the <a href="/wiki/Rift_Valley_Province" title="Rift Valley Province">Rift Valley</a>, the <a href="/wiki/Kalenjin_people" title="Kalenjin people">Kalenjin</a> have long made <a href="/wiki/Mursik" title="Mursik">mursik</a>, which they have with <i>kimyet</i> (<a href="/wiki/Ugali" title="Ugali">ugali</a>) and a vegetable relish called <i>isageek</i>. Also among the <a href="/wiki/Kikuyu_people" title="Kikuyu people">Kikuyu</a> of Central Kenya, many <a href="/wiki/Tubers" class="mw-redirect" title="Tubers">tubers</a>, including <i>ngwaci</i> (<a href="/wiki/Sweet_potatoes" class="mw-redirect" title="Sweet potatoes">sweet potatoes</a>), <i>ndũma</i> (<a href="/wiki/Taro_root" class="mw-redirect" title="Taro root">taro root</a>, known in Kenya as <a href="/wiki/Arrowroot" title="Arrowroot">arrowroot</a>), <i>ikwa</i> (<a href="/wiki/Yam_(vegetable)" title="Yam (vegetable)">yams</a>), and <i>mianga</i> (<a href="/wiki/Cassava" title="Cassava">cassava</a>) are eaten, as well as legumes like <a href="/wiki/Beans" class="mw-redirect" title="Beans">beans</a> and a Kikuyu bean known as <i>njahi</i>. Among the <a href="/wiki/Luo_people_of_Kenya_and_Tanzania" class="mw-redirect" title="Luo people of Kenya and Tanzania">Luo</a> residing on the western region around Lake Victoria, "kuon" (Ugali) and "rech" fish is a favourite, as well as "gweno" (chicken), "Aliya" (sun dried meat), "Onyoso" (type of ant), "Ng'wen" (termitoidae), "Dede" (grasshoppers), various birds and green vegetables "Alode" such as "Osuga", "Akeyo", "Muto", "Dodo", "Dek", "Apoth" and "Bo", all consumed with Ugali.
</p><p>As you travel around the country distinct differences are noted mainly based on what foods are locally available around such areas. Grains are a staple food for groups that grow grains (e.g. Kikuyu, Embu, Meru, Kisii, etc.). Other communities such as the Luo and the Coastal community have <a href="/wiki/Fish_(food)" class="mw-redirect" title="Fish (food)">fish</a> and seafood for their staple food as available in such areas. In semi-arid areas like Turkana, foods made from sorghum are more common staple foods. As you move towards the city – food eaten by working families vary according to preference and ethnicity. Rice and stew is more common with working families, and other dishes like <a href="/wiki/Chapati" title="Chapati">chapati</a>  which is a staple that originated from India and is similar to the flatbread (parantha), chicken stew, etc.
</p>''')

soup = kenyan_cuisine_raw

tags_to_extract = ['table', 'script', 'meta', 'style']
titles_to_extract = ['Jump up', 'Enlarge', re.compile("(Wikipedia:)"), "Help:Category", re.compile("(Category:Wiki.*)"), re.compile("(Category:Comm.*)"), re.compile("(Category:Articles.*)"), re.compile("(Category:All )"), 'ISBN_(identifier)', re.compile("(Special:)"), re.compile("(Category:CS1)")]
texts_to_extract = ['edit', re.compile("(Jump to)")]
classes_to_extract = ['reference']
ids_to_extract = ['mw-hidden-catlinks']

for t in tags_to_extract:
    [s.extract() for s in soup(t)]
for t in ids_to_extract:
    [s.extract() for s in soup(id=t)]
for t in titles_to_extract:
    [s.extract() for s in soup("a", title=t)]
for t in texts_to_extract:
    [s.extract() for s in soup("a", text=t)]
for t in classes_to_extract:
    [s.extract() for s in soup("a", class_=t)]

kenya_links = [("Kenyan#Cuisine", "Kenyan cuisine")] + list(set([(wikilink["href"].lstrip('/wiki/'), wikilink.text) for wikilink in soup.find_all("a", href=re.compile("(^\/wiki\/.+)")) if wikilink.text != "" and wikilink.text != " "]))
kenya_links

[('Kenyan#Cuisine', 'Kenyan cuisine'),
 ('Rift_Valley_Province', 'Rift Valley'),
 ('Beans', 'beans'),
 ('Sweet_potatoes', 'sweet potatoes'),
 ('Millet', 'millet'),
 ('Tubers', 'tubers'),
 ('Luhya_people', 'Luhya'),
 ('Chapati', 'chapati'),
 ('Kikuyu_people', 'Kikuyu'),
 ('Collard_greens', 'collards'),
 ('Yam_(vegetable)', 'yams'),
 ('Arrowroot', 'arrowroot'),
 ('Cassava', 'cassava'),
 ('Ugali', 'Ugali'),
 ('Kachumbari', 'kachumbari'),
 ('Taro_root', 'taro root'),
 ('Chicken_(food)', 'chicken'),
 ('Luo_people_of_Kenya_and_Tanzania', 'Luo'),
 ('Mursik', 'mursik'),
 ('Mchicha', 'mchicha'),
 ('Ugali', 'ugali'),
 ('African_Great_Lakes', 'African Great Lakes'),
 ('Sorghum', 'sorghum'),
 ('Sukuma_wiki', 'sukuma wiki'),
 ('Kale', 'kale'),
 ('Fish_(food)', 'fish'),
 ('Kalenjin_people', 'Kalenjin'),
 ('Swahili_language', 'Swahili')]

In [61]:
jersey_cuisine_raw = BeautifulSoup('''<div class="thumb tright"><div class="thumbinner" style="width:222px;"><a href="/wiki/File:Jersey_wonders.jpg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/1/14/Jersey_wonders.jpg/220px-Jersey_wonders.jpg" decoding="async" width="220" height="172" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/1/14/Jersey_wonders.jpg/330px-Jersey_wonders.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/1/14/Jersey_wonders.jpg/440px-Jersey_wonders.jpg 2x" data-file-width="1402" data-file-height="1098"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Jersey_wonders.jpg" class="internal" title="Enlarge"></a></div><i>Jersey wonders</i>, or <i>mèrvelles</i>, are a favourite snack consisting of fried dough, found especially at country fêtes. According to tradition, the success of cooking depends on the state of the tide.</div></div></div><p>Seafood has traditionally been important to the cuisine of Jersey: <a href="/wiki/Mussel" title="Mussel">mussels</a> (called <i>moules</i> in the island), <a href="/wiki/Oyster" title="Oyster">oysters</a>, <a href="/wiki/European_lobster" class="mw-redirect" title="European lobster">lobster</a> and crabs – especially <a href="/wiki/Maja_squinado" title="Maja squinado">spider crabs</a> – <a href="/wiki/Abalone" title="Abalone">ormers</a> and <a href="/wiki/Conger" title="Conger">conger</a>.
</p><p>Jersey milk being very rich, <a href="/wiki/Cream" title="Cream">cream</a> and butter have played a large part in insular cooking. <i>(See <a href="/wiki/Channel_Island_milk" title="Channel Island milk">Channel Island milk</a>)</i> However, there is no indigenous tradition of cheese making, contrary to the custom of mainland Normandy, but some cheese is produced commercially. Jersey <a href="/wiki/Fudge" title="Fudge">fudge</a>, mostly imported and made with milk from overseas Jersey cattle herds, is a popular food product with tourists.
</p><p><a href="/wiki/Jersey_Royal_potatoes" class="mw-redirect" title="Jersey Royal potatoes">Jersey Royal potatoes</a> are the local variety of <a href="/wiki/New_potato" class="mw-redirect" title="New potato">new potato</a>, and the island is famous for its early crop of Chats (small potatoes) from the south-facing côtils (steeply sloping fields). They were originally grown using <a href="/wiki/Seaweed_fertiliser" title="Seaweed fertiliser">vraic</a> as a natural fertiliser giving them their own individual taste, only a small portion of those grown in the island still use this method. They are eaten in a variety of ways, often simply boiled and served with butter or when not as fresh fried in butter.
</p><p>Apples historically were an important crop. <i>Bourdélots</i> are apple dumplings, but the most typical speciality is black butter (<i><a href="/wiki/Ni%C3%A8r_beurre" title="Nièr beurre">lé nièr beurre</a></i>), a dark spicy spread prepared from apples, cider and spices. <a href="/wiki/Cider" title="Cider">Cider</a> used to be an important export. After decline and near-disappearance in the late 20th century, apple production is being increased and promoted. Besides cider, <a href="/wiki/Apple_brandy" class="mw-redirect" title="Apple brandy">apple brandy</a> is produced. Other production of alcohol drinks includes wine,<sup id="cite_ref-105" class="reference"><a href="#cite_note-105">[105]</a></sup>  and in 2013 the first commercial <a href="/wiki/Vodka" title="Vodka">vodkas</a> made from Jersey Royal potatoes were marketed.<sup id="cite_ref-106" class="reference"><a href="#cite_note-106">[106]</a></sup>
</p><p>Among other traditional dishes are cabbage loaf, Jersey wonders (<i>les mèrvelles</i>), fliottes, bean crock (<i>les pais au fou</i>), <a href="/wiki/Urtica_dioica" title="Urtica dioica">nettle</a> (<i>ortchie</i>) soup, vraic buns.
</p>''')

soup = jersey_cuisine_raw

tags_to_extract = ['table', 'script', 'meta', 'style']
titles_to_extract = ['Jump up', 'Enlarge', re.compile("(Wikipedia:)"), "Help:Category", re.compile("(Category:Wiki.*)"), re.compile("(Category:Comm.*)"), re.compile("(Category:Articles.*)"), re.compile("(Category:All )"), 'ISBN_(identifier)', re.compile("(Special:)"), re.compile("(Category:CS1)")]
texts_to_extract = ['edit', re.compile("(Jump to)")]
classes_to_extract = ['reference']
ids_to_extract = ['mw-hidden-catlinks']

for t in tags_to_extract:
    [s.extract() for s in soup(t)]
for t in ids_to_extract:
    [s.extract() for s in soup(id=t)]
for t in titles_to_extract:
    [s.extract() for s in soup("a", title=t)]
for t in texts_to_extract:
    [s.extract() for s in soup("a", text=t)]
for t in classes_to_extract:
    [s.extract() for s in soup("a", class_=t)]

jersey_links = [("Jersey#Food_and_drink", "Jersey cuisine")] + list(set([(wikilink["href"].lstrip('/wiki/'), wikilink.text) for wikilink in soup.find_all("a", href=re.compile("(^\/wiki\/.+)")) if wikilink.text != "" and wikilink.text != " "]))
jersey_links

[('Jersey#Food_and_drink', 'Jersey cuisine'),
 ('Oyster', 'oysters'),
 ('Channel_Island_milk', 'Channel Island milk'),
 ('Seaweed_fertiliser', 'vraic'),
 ('Jersey_Royal_potatoes', 'Jersey Royal potatoes'),
 ('Mussel', 'mussels'),
 ('Vodka', 'vodkas'),
 ('Conger', 'conger'),
 ('Cider', 'Cider'),
 ('Maja_squinado', 'spider crabs'),
 ('Cream', 'cream'),
 ('Fudge', 'fudge'),
 ('European_lobster', 'lobster'),
 ('Apple_brandy', 'apple brandy'),
 ('Ni%C3%A8r_beurre', 'lé nièr beurre'),
 ('Abalone', 'ormers'),
 ('Urtica_dioica', 'nettle'),
 ('New_potato', 'new potato')]

In [62]:
cook_islands_cuisine_raw = BeautifulSoup('''<p>Due to the island location and the fact that the Cook Islands produce a significant array of fruits and vegetables, natural local produce, especially coconut, features in many of the dishes of the islands as does fresh seafood. While most food is imported from New Zealand, there are several Growers' Associations, such as Mangaian, Ngatangiia, Penrhyn, Puaikura, and Rakahanga, which contribute produce for local cuisine.<sup id="cite_ref-FOOTNOTECrocombe199077_26-0" class="reference"><a href="#cite_note-FOOTNOTECrocombe199077-26">[26]</a></sup> Typical local cuisine includes <a href="/wiki/Arrowroot" title="Arrowroot">arrowroot</a>, <a href="/wiki/Clam" title="Clam">clams</a>, octopus, and <a href="/wiki/Taro" title="Taro">taro</a>, and seasonings such as fresh <a href="/wiki/Ginger" title="Ginger">ginger</a>, <a href="/wiki/Lime_(fruit)" title="Lime (fruit)">lime</a>, lemon, <a href="/wiki/Basil" title="Basil">basil</a>, garlic and <a href="/wiki/Coconut" title="Coconut">coconut</a>.<sup id="cite_ref-FOOTNOTEPignéguyRaela2011_27-0" class="reference"><a href="#cite_note-FOOTNOTEPignéguyRaela2011-27">[27]</a></sup> <i>Rukau</i> is a dish of taro leaves cooked with coconut sauce and onion.  A meal of octopus is known locally as <i>Eke</i>, and <a href="/wiki/Suckling_pig" title="Suckling pig">suckling pig</a> is known as <i>Puaka</i>.<sup id="cite_ref-FOOTNOTEHunt200346_28-0" class="reference"><a href="#cite_note-FOOTNOTEHunt200346-28">[28]</a></sup> <i>Ika mata</i> is a dish of raw fish marinated with lemon or lime and served with coconut cream,<sup id="cite_ref-FOOTNOTEHunt200346_28-1" class="reference"><a href="#cite_note-FOOTNOTEHunt200346-28">[28]</a></sup> while <i>Pai Ika</i> and <i>Keke Ika</i> are also local fish fare.<sup id="cite_ref-FOOTNOTEMatenga-Smith199077_29-0" class="reference"><a href="#cite_note-FOOTNOTEMatenga-Smith199077-29">[29]</a></sup> <i>Poke</i> is a dessert which can be made in one of two ways, either with banana and coconut milk or with <a href="/wiki/Papaya" title="Papaya">pawpaw</a>.<sup id="cite_ref-FOOTNOTEFoldPritchard200586_30-0" class="reference"><a href="#cite_note-FOOTNOTEFoldPritchard200586-30">[30]</a></sup> <a href="/wiki/Soursop" title="Soursop">Soursop</a>, oranges and mangos are popular as juices.<sup id="cite_ref-FOOTNOTEMatenga-Smith199082_31-0" class="reference"><a href="#cite_note-FOOTNOTEMatenga-Smith199082-31">[31]</a></sup> <a href="/wiki/Coconut_water" title="Coconut water">Coconut water</a>, local beer (Cooks Lager), and coffee are popular beverages among the Cook Islanders.<sup id="cite_ref-FOOTNOTEHunt200346_28-2" class="reference"><a href="#cite_note-FOOTNOTEHunt200346-28">[28]</a></sup>
</p>''')

soup = cook_islands_cuisine_raw

tags_to_extract = ['table', 'script', 'meta', 'style']
titles_to_extract = ['Jump up', 'Enlarge', re.compile("(Wikipedia:)"), "Help:Category", re.compile("(Category:Wiki.*)"), re.compile("(Category:Comm.*)"), re.compile("(Category:Articles.*)"), re.compile("(Category:All )"), 'ISBN_(identifier)', re.compile("(Special:)"), re.compile("(Category:CS1)")]
texts_to_extract = ['edit', re.compile("(Jump to)")]
classes_to_extract = ['reference']
ids_to_extract = ['mw-hidden-catlinks']

for t in tags_to_extract:
    [s.extract() for s in soup(t)]
for t in ids_to_extract:
    [s.extract() for s in soup(id=t)]
for t in titles_to_extract:
    [s.extract() for s in soup("a", title=t)]
for t in texts_to_extract:
    [s.extract() for s in soup("a", text=t)]
for t in classes_to_extract:
    [s.extract() for s in soup("a", class_=t)]

cook_islands_links = [("Culture_of_the_Cook_Islands#Cuisine", "Cook Islands cuisine")] + list(set([(wikilink["href"].lstrip('/wiki/'), wikilink.text) for wikilink in soup.find_all("a", href=re.compile("(^\/wiki\/.+)")) if wikilink.text != "" and wikilink.text != " "]))
cook_islands_links

[('Culture_of_the_Cook_Islands#Cuisine', 'Cook Islands cuisine'),
 ('Taro', 'taro'),
 ('Basil', 'basil'),
 ('Lime_(fruit)', 'lime'),
 ('Clam', 'clams'),
 ('Arrowroot', 'arrowroot'),
 ('Papaya', 'pawpaw'),
 ('Soursop', 'Soursop'),
 ('Coconut', 'coconut'),
 ('Coconut_water', 'Coconut water'),
 ('Ginger', 'ginger'),
 ('Suckling_pig', 'suckling pig')]

In [63]:
tongan_cuisine_raw = BeautifulSoup('''<div class="thumb tright"><div class="thumbinner" style="width:202px;"><a href="/wiki/File:Tupenu02.JPG" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/en/thumb/9/95/Tupenu02.JPG/200px-Tupenu02.JPG" decoding="async" width="200" height="189" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/en/thumb/9/95/Tupenu02.JPG/300px-Tupenu02.JPG 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/9/95/Tupenu02.JPG/400px-Tupenu02.JPG 2x" data-file-width="1008" data-file-height="952"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Tupenu02.JPG" class="internal" title="Enlarge"></a></div>Women cooking <i>topai</i> for mourners.</div></div></div><p>In former times, there was only one main meal, a midday meal cooked in an earth oven. Villagers would rise, eat some leftover food from the previous day's meal, and set out to work in the fields, <a href="/wiki/Fishing" title="Fishing">fishing</a>, gathering <a href="/wiki/Shellfish" title="Shellfish">shellfish</a>, etc. The results of the morning's work would be cooked by the men, and served to the assembled household. The remnants would be placed in a basket suspended from a tree. This food is served as an end-of-the-day snack as well as the next day's breakfast. Food past its prime was given to the pigs.
</p><p>The diet consisted mainly of <a href="/wiki/Taro" title="Taro">taro</a>, <a href="/wiki/Oxalis_tuberosa" title="Oxalis tuberosa">yams</a>, <a href="/wiki/Bananas" class="mw-redirect" title="Bananas">bananas</a>, <a href="/wiki/Coconuts" class="mw-redirect" title="Coconuts">coconuts</a>, and <a href="/wiki/Fish" title="Fish">fish</a> baked in leaves; shellfish were usually served raw, as a <a href="/wiki/Relish" title="Relish">relish</a>. The liquid from the center of coconuts was commonly drunk, and the soft "spoon meat" of young coconuts much relished. Baked <a href="/wiki/Breadfruit" title="Breadfruit">breadfruit</a> was eaten in season. <a href="/wiki/Pigs" class="mw-redirect" title="Pigs">Pigs</a> were killed and cooked only on special occasions, such as <a href="/wiki/Weddings" class="mw-redirect" title="Weddings">weddings</a>, <a href="/wiki/Funerals" class="mw-redirect" title="Funerals">funerals</a>, <a href="/wiki/Feasts" class="mw-redirect" title="Feasts">feasts</a> honoring a visiting chief, and the like. Tongans also ate <a href="/wiki/Chickens" class="mw-redirect" title="Chickens">chickens</a>.
</p><p>Food could be stored by feeding it to pigs. Pre-contact Tongans also built elevated storehouses for yams. Yams would keep only a few months. Hence a household's main security was generous distribution of food to relatives and neighbors, who were thus put under an obligation to share in their turn.
</p><p>Many new foods were introduced in the 19th and early 20th centuries, following Western contacts and settlements. The <a href="/wiki/Cassava" title="Cassava">cassava</a> plant was one such introduction; it is called <i>manioke</i> in Tongan. While it lacks the prestige of the yam, it is an easy plant to grow and a common crop. Introduced <a href="/wiki/Watermelons" class="mw-redirect" title="Watermelons">watermelons</a> became popular. They were eaten either by themselves, or pulped and mixed with <a href="/wiki/Coconut_milk" title="Coconut milk">coconut milk</a>, forming a popular drink called <i><a href="/wiki/%27otai" title="'otai">'otai</a></i>. Other fruits, such as <a href="/wiki/Orange_(fruit)" title="Orange (fruit)">oranges</a>, <a href="/wiki/Lemon" title="Lemon">lemons</a>, and <a href="/wiki/Lime_(fruit)" title="Lime (fruit)">limes</a>, became popular. Tongans also adopted <a href="/wiki/Onion" title="Onion">onions</a>, <a href="/wiki/Scallion" title="Scallion">green onions</a>, <a href="/wiki/Cabbage" title="Cabbage">cabbage</a>, <a href="/wiki/Carrot" title="Carrot">carrots</a>, <a href="/wiki/Tomato" title="Tomato">tomatoes</a>, and other common vegetables. In the last few decades, Tongan farmers with access to large tracts of land have engaged in commercial farming of <a href="/wiki/Pumpkin" title="Pumpkin">pumpkins</a> and other easily shipped vegetables as cash crops.
</p><p>Tongans now consume large quantities of imported <a href="/wiki/Flour" title="Flour">flour</a> and <a href="/wiki/Sugar" title="Sugar">sugar</a>. One dish that uses both is <i>topai</i> (doughboys), flour and water worked into a paste and dropped into a kettle of boiling water, then served with a <a href="/wiki/Syrup" title="Syrup">syrup</a> of sugar and coconut milk. Topai are a common funeral food, being easily prepared for hundreds of mourners.
</p><p>There are now bakeries in the larger cities. The most popular loaves are soft, white, and bland. There are also local soft drink bottlers, who make various local varieties of <a href="/wiki/Soft_drink" title="Soft drink">soda</a>. A Tongan who might once have breakfasted on bits of cooked <a href="/wiki/Pork" title="Pork">pork</a> and yam from a hanging basket may now have <a href="/wiki/White_bread" title="White bread">white bread</a> and soda for breakfast.
</p><p>Purchased prepared foods have also made great headway, even in remote villages. Canned <a href="/wiki/Cornbeef" class="mw-redirect" title="Cornbeef">cornbeef</a> is a great favorite. It is eaten straight from the can, or mixed with coconut milk and onions, wrapped in leaves, and baked in the earth oven. Tongans also eat canned fish, such as <a href="/wiki/Tuna" title="Tuna">tuna</a>. In villages or towns with refrigeration, cheap frozen "<a href="/wiki/Mutton_flaps" title="Mutton flaps">mutton flaps</a>" imported from New Zealand are popular. Tongans also eat the common South Pacific "ship's biscuit", hard plain <a href="/wiki/Cracker_(food)" title="Cracker (food)">crackers</a> once a shipboard staple. These crackers are called <i>mā pakupaku</i>.
</p><p>Tongans no longer make an earth oven every day. Most daily cooking is done by women, who cook in battered pots over open fires in the village, in wood-burning stoves in some households, and on gas or electric ranges in some of the larger towns. The meal schedule has also changed, to more Westernized breakfast, light lunch, and heavy dinner. Tongans say that the old schedule is unworkable when household members have Western-style jobs, or attend schools at some distance from home; such family members cannot come home to eat, then have a doze after a heavy mid-day meal.
</p><p>As well as drinking soda, Tongans now drink <a href="/wiki/Tea" title="Tea">tea</a> and <a href="/wiki/Coffee" title="Coffee">coffee</a>. Usually this is of the cheapest variety, and served with tinned <a href="/wiki/Condensed_milk" title="Condensed milk">condensed milk</a>.
</p><p>Some men drink <a href="/wiki/Alcohol_(drug)" title="Alcohol (drug)">alcohol</a>. Sometimes this is imported Australian or New Zealand <a href="/wiki/Beer" title="Beer">beer</a>; more often it is home-brew, <i><a href="/wiki/Hopi" title="Hopi">hopi</a></i>, made with <a href="/wiki/Water" title="Water">water</a>, sugar or mashed <a href="/wiki/Fruit" title="Fruit">fruit</a>, and <a href="/wiki/Yeast" title="Yeast">yeast</a>. Imported drinks are sold only to Tongans who have liquor permits, which require a visit to a government office, and limit the amount of alcohol which can be purchased. There are no such formalities with <i>hopi</i>. Drinking is usually done secretively; a group of men gather and drink until they are drunk. Such gatherings sometimes result in drunken quarrels and assaults.
</p>''')

soup = tongan_cuisine_raw

tags_to_extract = ['table', 'script', 'meta', 'style']
titles_to_extract = ['Jump up', 'Enlarge', re.compile("(Wikipedia:)"), "Help:Category", re.compile("(Category:Wiki.*)"), re.compile("(Category:Comm.*)"), re.compile("(Category:Articles.*)"), re.compile("(Category:All )"), 'ISBN_(identifier)', re.compile("(Special:)"), re.compile("(Category:CS1)")]
texts_to_extract = ['edit', re.compile("(Jump to)")]
classes_to_extract = ['reference']
ids_to_extract = ['mw-hidden-catlinks']

for t in tags_to_extract:
    [s.extract() for s in soup(t)]
for t in ids_to_extract:
    [s.extract() for s in soup(id=t)]
for t in titles_to_extract:
    [s.extract() for s in soup("a", title=t)]
for t in texts_to_extract:
    [s.extract() for s in soup("a", text=t)]
for t in classes_to_extract:
    [s.extract() for s in soup("a", class_=t)]

tonga_links = [("Culture_of_Tonga#Cuisine", "Tongan cuisine")] + list(set([(wikilink["href"].lstrip('/wiki/'), wikilink.text) for wikilink in soup.find_all("a", href=re.compile("(^\/wiki\/.+)")) if wikilink.text != "" and wikilink.text != " "]))
tonga_links

[('Culture_of_Tonga#Cuisine', 'Tongan cuisine'),
 ('Breadfruit', 'breadfruit'),
 ('Oxalis_tuberosa', 'yams'),
 ('Yeast', 'yeast'),
 ('Lemon', 'lemons'),
 ('Feasts', 'feasts'),
 ('Mutton_flaps', 'mutton flaps'),
 ('Soft_drink', 'soda'),
 ('White_bread', 'white bread'),
 ('Fruit', 'fruit'),
 ('Lime_(fruit)', 'limes'),
 ('Scallion', 'green onions'),
 ('Carrot', 'carrots'),
 ('Syrup', 'syrup'),
 ('Chickens', 'chickens'),
 ('Watermelons', 'watermelons'),
 ('Weddings', 'weddings'),
 ('Sugar', 'sugar'),
 ('Relish', 'relish'),
 ('Taro', 'taro'),
 ('Pigs', 'Pigs'),
 ('Tea', 'tea'),
 ('Coconuts', 'coconuts'),
 ('Cassava', 'cassava'),
 ('Cabbage', 'cabbage'),
 ('Tomato', 'tomatoes'),
 ('Bananas', 'bananas'),
 ('Orange_(fruit)', 'oranges'),
 ('Condensed_milk', 'condensed milk'),
 ('Coffee', 'coffee'),
 ('Alcohol_(drug)', 'alcohol'),
 ('Onion', 'onions'),
 ('%27otai', "'otai"),
 ('Fish', 'fish'),
 ('Pork', 'pork'),
 ('Water', 'water'),
 ('Tuna', 'tuna'),
 ('Cracker_(food)', 'crackers'),
 ('Funerals

In [64]:
st_kitts_cuisine_raw = BeautifulSoup('''<p>With its rich soil, St. Kitts and Nevis grow a wide variety of fresh <a href="/wiki/Produce" title="Produce">produce</a>.  Abundant <a href="/wiki/Seafood" title="Seafood">seafood</a> and <a href="/wiki/Meat" title="Meat">meats</a> such as <a href="/wiki/Goat" title="Goat">goat</a> add to the diet.  The style of cooking is fairly simple, flavored much like other <a href="/wiki/West_Indian" title="West Indian">West Indian</a> cuisine.  Goat water <a href="/wiki/Stew" title="Stew">stew</a>, perhaps the country's most well-known dish, mixes goat, <a href="/wiki/Breadfruit" title="Breadfruit">breadfruit</a>, green pawpaw (<a href="/wiki/Papaya" title="Papaya">papaya</a>), and <a href="/wiki/Dumpling" title="Dumpling">dumplings</a> (also known as "droppers") in a tomato-based stew. Another favorite dish is cook-up, or <a href="/wiki/Pelau" title="Pelau">pelau</a>, which combines <a href="/wiki/Chicken" title="Chicken">chicken</a>, pig tail, <a href="/wiki/Saltfish" class="mw-redirect" title="Saltfish">saltfish</a> and <a href="/wiki/Vegetable" title="Vegetable">vegetables</a> with <a href="/wiki/Rice" title="Rice">rice</a> and <a href="/wiki/Pigeon_pea" title="Pigeon pea">pigeon peas</a>.  Conkies bear a large similarity to <a href="/wiki/Tamales" class="mw-redirect" title="Tamales">tamales</a>, though instead of having filling rolled inside the dough, the cornmeal is mixed together with grated sweet potato, pumpkin, coconut, and a few other ingredients; after wrapping the dough in banana leaves, they're boiled rather than steamed. Sweets tend to be simply made, sometimes with nothing more than fruit, like tamarind or guava, and sugar.
</p><p><a href="/wiki/Rum" title="Rum">Rum</a> is as popular on St. Kitts and Nevis as it is throughout the Caribbean.  The Brinley Gold Company manufactures rum on St. Kitts, with such distinctive flavors as coffee, mango, and vanilla.  But the national drink is actually Cane Spirits Rothschild (often abbreviated to CSR), distilled from fresh sugar cane. Belmont Estate and St. Kitts Rum also make rum on the island. In addition several of the beach bars will provide moonshine rum produced by individuals with homemade stills. 
</p><p>Many villages on Nevis hold cookouts on Friday and Saturday nights, where people come together to eat, drink, play games like <a href="/wiki/Dominoes" title="Dominoes">dominoes</a>, and have a good time.
</p>''')

soup = st_kitts_cuisine_raw

tags_to_extract = ['table', 'script', 'meta', 'style']
titles_to_extract = ['Jump up', 'Enlarge', re.compile("(Wikipedia:)"), "Help:Category", re.compile("(Category:Wiki.*)"), re.compile("(Category:Comm.*)"), re.compile("(Category:Articles.*)"), re.compile("(Category:All )"), 'ISBN_(identifier)', re.compile("(Special:)"), re.compile("(Category:CS1)")]
texts_to_extract = ['edit', re.compile("(Jump to)")]
classes_to_extract = ['reference']
ids_to_extract = ['mw-hidden-catlinks']

for t in tags_to_extract:
    [s.extract() for s in soup(t)]
for t in ids_to_extract:
    [s.extract() for s in soup(id=t)]
for t in titles_to_extract:
    [s.extract() for s in soup("a", title=t)]
for t in texts_to_extract:
    [s.extract() for s in soup("a", text=t)]
for t in classes_to_extract:
    [s.extract() for s in soup("a", class_=t)]

st_kitts_links = [("Culture_of_Saint_Kitts_and_Nevis#Cuisine", "Cuisine of Saint Kitts and Nevis")] + list(set([(wikilink["href"].lstrip('/wiki/'), wikilink.text) for wikilink in soup.find_all("a", href=re.compile("(^\/wiki\/.+)")) if wikilink.text != "" and wikilink.text != " "]))
st_kitts_links

[('Culture_of_Saint_Kitts_and_Nevis#Cuisine',
  'Cuisine of Saint Kitts and Nevis'),
 ('Seafood', 'seafood'),
 ('Dumpling', 'dumplings'),
 ('Stew', 'stew'),
 ('Chicken', 'chicken'),
 ('Goat', 'goat'),
 ('Meat', 'meats'),
 ('Pigeon_pea', 'pigeon peas'),
 ('Papaya', 'papaya'),
 ('Rice', 'rice'),
 ('Vegetable', 'vegetables'),
 ('Breadfruit', 'breadfruit'),
 ('Produce', 'produce'),
 ('Tamales', 'tamales'),
 ('Pelau', 'pelau'),
 ('West_Indian', 'West Indian'),
 ('Dominoes', 'dominoes'),
 ('Saltfish', 'saltfish'),
 ('Rum', 'Rum')]

In [65]:
martinique_cuisine_raw = BeautifulSoup('''<p>As one would expect, <a href="/wiki/French_cuisine" title="French cuisine">French</a> and <a href="/wiki/Caribbean_cuisine" title="Caribbean cuisine">Creole cuisine</a> dominate Martinique's culinary landscape.  The two styles also combine by using French techniques with local produce, such as <a href="/wiki/Breadfruit" title="Breadfruit">breadfruit</a>, <a href="/wiki/Cassava" title="Cassava">cassava</a>, and <a href="/wiki/Christophine" class="mw-redirect" title="Christophine">christophine</a> (chayote).  Creole dishes rely heavily on <a href="/wiki/Seafood" title="Seafood">seafood</a>, including <a href="/wiki/Curry" title="Curry">curries</a> and <a href="/wiki/Fritter" title="Fritter">fritters</a>.  An exception is <a href="/wiki/Boudin" title="Boudin">Boudin</a>, a Creole type of blood sausage.  A dash of Chien sauce (made from onions, shallots, peppers, oil, and vinegar) adds a spicy touch to meals.  The favored island drink, 'Ti punch,' is a mixture of five parts of white rum to one part <a href="/wiki/Sugarcane" title="Sugarcane">sugarcane</a> syrup.  <a href="/wiki/Cr%C3%AAperie" class="mw-redirect" title="Crêperie">Crêperies</a>, <a href="/wiki/Brasserie" title="Brasserie">brasseries</a>, and restaurants featuring cuisine from various French regions can be found all over Martinique.
</p>''')

soup = martinique_cuisine_raw

tags_to_extract = ['table', 'script', 'meta', 'style']
titles_to_extract = ['Jump up', 'Enlarge', re.compile("(Wikipedia:)"), "Help:Category", re.compile("(Category:Wiki.*)"), re.compile("(Category:Comm.*)"), re.compile("(Category:Articles.*)"), re.compile("(Category:All )"), 'ISBN_(identifier)', re.compile("(Special:)"), re.compile("(Category:CS1)")]
texts_to_extract = ['edit', re.compile("(Jump to)")]
classes_to_extract = ['reference']
ids_to_extract = ['mw-hidden-catlinks']

for t in tags_to_extract:
    [s.extract() for s in soup(t)]
for t in ids_to_extract:
    [s.extract() for s in soup(id=t)]
for t in titles_to_extract:
    [s.extract() for s in soup("a", title=t)]
for t in texts_to_extract:
    [s.extract() for s in soup("a", text=t)]
for t in classes_to_extract:
    [s.extract() for s in soup("a", class_=t)]

martinique_links = [("Culture_of_Martinique#Cuisine", "Cuisine of Martinique")] + list(set([(wikilink["href"].lstrip('/wiki/'), wikilink.text) for wikilink in soup.find_all("a", href=re.compile("(^\/wiki\/.+)")) if wikilink.text != "" and wikilink.text != " "]))
martinique_links

[('Culture_of_Martinique#Cuisine', 'Cuisine of Martinique'),
 ('Seafood', 'seafood'),
 ('Curry', 'curries'),
 ('French_cuisine', 'French'),
 ('Fritter', 'fritters'),
 ('Sugarcane', 'sugarcane'),
 ('Cr%C3%AAperie', 'Crêperies'),
 ('Boudin', 'Boudin'),
 ('Christophine', 'christophine'),
 ('Cassava', 'cassava'),
 ('Breadfruit', 'breadfruit'),
 ('Brasserie', 'brasseries'),
 ('Caribbean_cuisine', 'Creole cuisine')]

In [91]:
def get_links_from_raw_html(raw_html, url_end, area):

    soup = BeautifulSoup(raw_html)

    tags_to_extract = ['table', 'script', 'meta', 'style']
    titles_to_extract = ['Jump up', 'Enlarge', re.compile("(Wikipedia:)"), "Help:Category", re.compile("(Category:Wiki.*)"), re.compile("(Category:Comm.*)"), re.compile("(Category:Articles.*)"), re.compile("(Category:All )"), 'ISBN_(identifier)', re.compile("(Special:)"), re.compile("(Category:CS1)")]
    texts_to_extract = ['edit', re.compile("(Jump to)")]
    classes_to_extract = ['reference']
    ids_to_extract = ['mw-hidden-catlinks']

    for t in tags_to_extract:
        [s.extract() for s in soup(t)]
    for t in ids_to_extract:
        [s.extract() for s in soup(id=t)]
    for t in titles_to_extract:
        [s.extract() for s in soup("a", title=t)]
    for t in texts_to_extract:
        [s.extract() for s in soup("a", text=t)]
    for t in classes_to_extract:
        [s.extract() for s in soup("a", class_=t)]

    links = [(url_end, area)] + list(set([(wikilink["href"].lstrip('/wiki/'), wikilink.text) for wikilink in soup.find_all("a", href=re.compile("(^\/wiki\/.+)")) if wikilink.text != "" and wikilink.text != " "]))
    
    return links

In [ ]:
central_africa = ["Angola", "Cameroon", "Central African Republic", "Chad", "Democratic Republic of the Congo", "Republic of the Congo", "Equatorial Guinea", "Gabon", "São Tomé and Príncipe"]


In [92]:
raw_html = '''<div class="thumb tright"><div class="thumbinner" style="width:302px;"><a href="/wiki/File:Madesu.jpg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/c/cb/Madesu.jpg/300px-Madesu.jpg" decoding="async" width="300" height="244" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/c/cb/Madesu.jpg/450px-Madesu.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/c/cb/Madesu.jpg/600px-Madesu.jpg 2x" data-file-width="1242" data-file-height="1012"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Madesu.jpg" class="internal" title="Enlarge"></a></div><i><a href="/w/index.php?title=Madesu&amp;action=edit&amp;redlink=1" class="new" title="Madesu (page does not exist)">Madesu</a></i> beans are a staple in Central Africa</div></div></div><p><br>
<a href="/wiki/Central_Africa" title="Central Africa">Central Africa</a> stretches from the <a href="/wiki/Tibesti_Mountains" title="Tibesti Mountains">Tibesti Mountains</a> in the north to the vast <a href="/wiki/Rainforest" title="Rainforest">rainforest</a> basin of the <a href="/wiki/Congo_River" title="Congo River">Congo River</a>, the highlands of Kivu and the savana of Katanga. This region has received culinary influence of the <a href="/wiki/Swahili_people" title="Swahili people">Swahilis</a> (a culture that evolved via the combination of Bantu, Yemeni, Omani and Indian cultures) during the East African Slave Trade. Swahili culinary influences can be found in dishes such as: mandanzi, pilaf rice, kachumbari, <a href="/wiki/Samosa" title="Samosa">sambsusa</a>, kuku paka, etc.<sup id="cite_ref-6" class="reference"><a href="#cite_note-6">[6]</a></sup> Central African cuisine has also been influenced by the Portuguese, by way of the <a href="/wiki/Kingdom_of_Kongo" title="Kingdom of Kongo">Kongo</a> and <a href="/wiki/Ndongo" class="mw-redirect" title="Ndongo">Ndongo</a> Kingdoms. Salt fish was introduced following trade in the late XVIIth and the Kikongo term for salt fish, makayabu, comes from the term bacalhau (ba-cal-ha-u).<sup id="cite_ref-7" class="reference"><a href="#cite_note-7">[7]</a></sup> The Portuguese culinary influence is especially prominent in Angola, Sao Tomé and Equatorial Guinea. Central Africa has also been influenced by the cuisine of the regions East, West and Southern Africa because of their close proximity. I.e: Babuté/<a href="/wiki/Bobotie" title="Bobotie">Bobotie</a> is shared with the south, nyama choma with the east and gombos with West Africa.
</p><div class="thumb tright"><div class="thumbinner" style="width:302px;"><a href="/wiki/File:Ntaba.jpg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/f/fb/Ntaba.jpg/300px-Ntaba.jpg" decoding="async" width="300" height="302" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/f/fb/Ntaba.jpg/450px-Ntaba.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/f/fb/Ntaba.jpg 2x" data-file-width="599" data-file-height="602"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Ntaba.jpg" class="internal" title="Enlarge"></a></div><i>Ntaba</i> Central African-style lamb barbecue</div></div></div><p>The main ingredients are <a href="/wiki/Plantain_(cooking)" class="mw-redirect" title="Plantain (cooking)">plantains</a>, <a href="/wiki/Cassava" title="Cassava">cassava</a>, rice, kwanga (cassava dumpling) and <a href="/wiki/Yam_(vegetable)" title="Yam (vegetable)">yam</a>. <a href="/wiki/Fufu" title="Fufu">Fufu</a>-like starchy foods are usually made from fermented cassava roots, but they can also be made with plantain, corn maize and yam. Fufu is served buffet style with grilled meat, fish, stews, greens and <a href="/wiki/Pili_pili" class="mw-redirect" title="Pili pili">piment</a> . A variety of local ingredients are used while preparing other dishes like <a href="/wiki/Spinach" title="Spinach">spinach</a> stew cooked with tomato, peppers, chillis, onions, and peanut butter.<sup id="cite_ref-8" class="reference"><a href="#cite_note-8">[8]</a></sup> Eastern central Africa is also one of the few regions in Africa that uses potatoes as one of its main bases, since potatoes grow easily in the region.
</p><p>Cassava plants are also consumed as cooked <a href="/wiki/Leaf_vegetable" title="Leaf vegetable">greens</a>. Groundnut (peanut) stew is also prepared, containing <a href="/wiki/Chicken_(food)" class="mw-redirect" title="Chicken (food)">chicken</a>, <a href="/wiki/Okra" title="Okra">okra</a>, <a href="/wiki/Ginger" title="Ginger">ginger</a>, and other spices. Another commonly served dish is bambara, a <a href="/wiki/Porridge" title="Porridge">porridge</a> of rice, peanut butter and sugar. Chicken and fish are favorite meat dishes, but <a href="/wiki/Bushmeat" title="Bushmeat">game meat</a> preparations containing <a href="/wiki/Crocodile" title="Crocodile">crocodile</a>, <a href="/wiki/Elephant" title="Elephant">elephant</a>, <a href="/wiki/Antelope" title="Antelope">antelope</a> and <a href="/wiki/Warthog" title="Warthog">warthog</a> are also served occasionally.<sup id="cite_ref-Huchzermeyer_2003_p._130_9-0" class="reference"><a href="#cite_note-Huchzermeyer_2003_p._130-9">[9]</a></sup><sup id="cite_ref-Daniel_Stiles_p._36_10-0" class="reference"><a href="#cite_note-Daniel_Stiles_p._36-10">[10]</a></sup><sup id="cite_ref-Stiles_2011_p._25_11-0" class="reference"><a href="#cite_note-Stiles_2011_p._25-11">[11]</a></sup><sup id="cite_ref-Whitford_1877_p._212_12-0" class="reference"><a href="#cite_note-Whitford_1877_p._212-12">[12]</a></sup><sup id="cite_ref-Gibbons_1898_p._223_13-0" class="reference"><a href="#cite_note-Gibbons_1898_p._223-13">[13]</a></sup>
</p>'''

central_africa = ["Angola", "Cameroon", "Central African Republic", "Chad", "Democratic Republic of the Congo", "Republic of the Congo", "Equatorial Guinea", "Gabon", "São Tomé and Príncipe"]
central_african_links = get_links_from_raw_html(raw_html, "African_cuisine#Central_Africa", "Central Africa")

[('African_cuisine#Central_Africa', 'Central Africa'),
 ('Antelope', 'antelope'),
 ('Kingdom_of_Kongo', 'Kongo'),
 ('Yam_(vegetable)', 'yam'),
 ('Pili_pili', 'piment'),
 ('Congo_River', 'Congo River'),
 ('Crocodile', 'crocodile'),
 ('Swahili_people', 'Swahilis'),
 ('Spinach', 'spinach'),
 ('Fufu', 'Fufu'),
 ('Plantain_(cooking)', 'plantains'),
 ('Bobotie', 'Bobotie'),
 ('Ndongo', 'Ndongo'),
 ('Central_Africa', 'Central Africa'),
 ('Leaf_vegetable', 'greens'),
 ('Bushmeat', 'game meat'),
 ('Elephant', 'elephant'),
 ('Cassava', 'cassava'),
 ('Porridge', 'porridge'),
 ('Samosa', 'sambsusa'),
 ('Chicken_(food)', 'chicken'),
 ('Ginger', 'ginger'),
 ('Warthog', 'warthog'),
 ('Rainforest', 'rainforest'),
 ('Tibesti_Mountains', 'Tibesti Mountains'),
 ('Okra', 'okra')]

In [95]:
raw_html = '''<div class="thumb tright"><div class="thumbinner" style="width:202px;"><a href="/wiki/File:Ugali_and_cabbage.jpg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/f/fe/Ugali_and_cabbage.jpg/200px-Ugali_and_cabbage.jpg" decoding="async" width="200" height="175" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/f/fe/Ugali_and_cabbage.jpg/300px-Ugali_and_cabbage.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/f/fe/Ugali_and_cabbage.jpg/400px-Ugali_and_cabbage.jpg 2x" data-file-width="2183" data-file-height="1912"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Ugali_and_cabbage.jpg" class="internal" title="Enlarge"></a></div><i><a href="/wiki/Ugali" title="Ugali">Ugali</a></i> is pictured here with a side dish of <a href="/wiki/Cabbage" title="Cabbage">cabbage</a>, though it is more typically eaten with <a href="/wiki/Collard_greens" class="mw-redirect" title="Collard greens">collard greens</a> (<i>sukuma wiki</i>).</div></div></div><div class="thumb tright"><div class="thumbinner" style="width:222px;"><a href="/wiki/File:Forodhani_park_food_stand.jpg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Forodhani_park_food_stand.jpg/220px-Forodhani_park_food_stand.jpg" decoding="async" width="220" height="147" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Forodhani_park_food_stand.jpg/330px-Forodhani_park_food_stand.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Forodhani_park_food_stand.jpg/440px-Forodhani_park_food_stand.jpg 2x" data-file-width="1600" data-file-height="1067"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Forodhani_park_food_stand.jpg" class="internal" title="Enlarge"></a></div>Barbecued beef cubes and seafood in <a href="/wiki/Forodhani_Gardens" title="Forodhani Gardens">Forodhani Gardens</a>, <a href="/wiki/Zanzibar" title="Zanzibar">Zanzibar</a></div></div></div><div class="thumb tright"><div class="thumbinner" style="width:222px;"><a href="/wiki/File:Kitcha_fit_fit.png" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/b/b5/Kitcha_fit_fit.png/220px-Kitcha_fit_fit.png" decoding="async" width="220" height="231" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/b/b5/Kitcha_fit_fit.png/330px-Kitcha_fit_fit.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/b/b5/Kitcha_fit_fit.png/440px-Kitcha_fit_fit.png 2x" data-file-width="828" data-file-height="868"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Kitcha_fit_fit.png" class="internal" title="Enlarge"></a></div><i><a href="/wiki/Kitcha_fit-fit" class="mw-redirect" title="Kitcha fit-fit">Kitcha fit-fit</a></i>, a mainstay in <a href="/wiki/Eritrean_cuisine" title="Eritrean cuisine">Eritrean cuisine</a>, is presented with a scoop of fresh yoghurt and topped with berbere (spice).</div></div></div><p>The cuisine of <a href="/wiki/East_Africa" title="East Africa">East Africa</a> varies from area to area. In the inland savannah, the traditional cuisine of cattle-keeping peoples is distinctive in that meat products are generally absent. <a href="/wiki/Cattle" title="Cattle">Cattle</a>, <a href="/wiki/Sheep" title="Sheep">sheep</a>, <a href="/wiki/Pigs" class="mw-redirect" title="Pigs">pigs</a> and <a href="/wiki/Goat" title="Goat">goats</a> were regarded as a form of <a href="/wiki/Currency" title="Currency">currency</a><sup id="cite_ref-14" class="reference"><a href="#cite_note-14">[14]</a></sup> and a store of wealth, and are not generally consumed as food.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (October 2017)">citation needed</span></a></i>]</sup>
</p><p>In some areas, traditional <a href="/wiki/East_Africa" title="East Africa">East Africans</a> consume the milk and blood of cattle, but rarely the meat. Elsewhere, other peoples are farmers who grow a variety of grains and vegetables. <a href="/wiki/Maize" title="Maize">Maize</a> (corn) is the basis of <a href="/wiki/Ugali" title="Ugali">ugali</a>, the local version of West and Central Africa's <a href="/wiki/Fufu" title="Fufu">fufu</a>. Ugali is a starch dish eaten with meats or stews. In <a href="/wiki/Uganda" title="Uganda">Uganda</a>, steamed green <a href="/wiki/Banana" title="Banana">bananas</a> called <a href="/wiki/Matoke" class="mw-redirect" title="Matoke">matoke</a> provide the starch filler of many meals.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (October 2017)">citation needed</span></a></i>]</sup>
</p><p>Around 1000&nbsp;years ago, <a href="/wiki/Oman" title="Oman">Omani</a> and <a href="/wiki/Yemen" title="Yemen">Yemeni</a> merchants settled on the <a href="/wiki/Swahili_Coast" class="mw-redirect" title="Swahili Coast">Swahili Coast</a>. Middle Eastern influences are especially reflected in the <a href="/wiki/Swahili_culture" title="Swahili culture">Swahili</a> cuisine of the coast – steamed or cooked rice with spices in <a href="/wiki/Persian_people" class="mw-redirect" title="Persian people">Persian</a> style; <a href="/wiki/Saffron" title="Saffron">saffron</a>, <a href="/wiki/Clove" title="Clove">cloves</a>, <a href="/wiki/Cinnamon" title="Cinnamon">cinnamon</a> and several other spices; and <a href="/wiki/Pomegranate" title="Pomegranate">pomegranate</a> juice.<sup id="cite_ref-15" class="reference"><a href="#cite_note-15">[15]</a></sup>
</p><p>Several centuries later, the British and the Indians came, and both brought with them foods such as Indian spiced vegetable <a href="/wiki/Curries" class="mw-redirect" title="Curries">curries</a>, <a href="/wiki/Lentil" title="Lentil">lentil</a> <a href="/wiki/Soup" title="Soup">soups</a>, chapattis and a variety of <a href="/wiki/Pickling" title="Pickling">pickles</a> which have influenced various local dishes. Some common ingredients used in this region include oranges, lemons, limes, chillis, capsicum peppers, maize, tomatoes, and strawberries. 
</p><p>In the Horn of Africa, the main traditional dishes in <a href="/wiki/Eritrean_cuisine" title="Eritrean cuisine">Eritrean cuisine</a> and <a href="/wiki/Ethiopian_cuisine" title="Ethiopian cuisine">Ethiopian cuisine</a> are <i><a href="/wiki/Wat_(food)" title="Wat (food)">tsebhis</a></i> (stews) served with <i><a href="/wiki/Injera" title="Injera">injera</a></i><sup id="cite_ref-Neo-Africanism_2008_2-1" class="reference"><a href="#cite_note-Neo-Africanism_2008-2">[2]</a></sup><sup id="cite_ref-webcitation_16-0" class="reference"><a href="#cite_note-webcitation-16">[16]</a></sup> (flatbread made from <a href="/wiki/Teff" title="Teff">teff</a>,<sup id="cite_ref-webcitation_16-1" class="reference"><a href="#cite_note-webcitation-16">[16]</a></sup> <a href="/wiki/Wheat" title="Wheat">wheat</a>, or <a href="/wiki/Sorghum" title="Sorghum">sorghum</a>) and <a href="/w/index.php?title=Hilbet&amp;action=edit&amp;redlink=1" class="new" title="Hilbet (page does not exist)">hilbet</a> (paste made from <a href="/wiki/Legume" title="Legume">legumes</a>, mainly <a href="/wiki/Lentil" title="Lentil">lentils</a> and <a href="/wiki/Vicia_faba" title="Vicia faba">fava beans</a>). Eritrean and Ethiopian cuisine (especially in the northern half) are very similar, given the shared history of the two countries.
</p><p>Eritrean and Ethiopian food habits vary regionally. In the <a href="/wiki/Ethiopian_Highlands" title="Ethiopian Highlands">highlands</a>, injera is the staple diet and is eaten daily among the <a href="/wiki/Tigray-Tigrinya_people" class="mw-redirect" title="Tigray-Tigrinya people">Tigrinya</a>. Injera is made out of teff, wheat, barley, sorghum or corn, and resembles a spongy, slightly sour pancake. When eating, diners generally share food from a large tray placed in the center of a low dining table. Numerous injera is layered on this tray and topped with various spicy stews. Diners then break into the section of injera in front of them, tearing off pieces and dipping them into the stews.
</p><p>In the lowlands, the main dish is <i><a href="/wiki/Akelet" class="mw-redirect" title="Akelet">akelet</a></i>, a porridge-like dish made from wheat flour dough. A <a href="/wiki/Ladle_(spoon)" title="Ladle (spoon)">ladle</a> is used to scoop out the top, which is filled with berbere and butter sauce and surrounded by milk or yoghurt. A small piece of dough is broken and then used to scoop up the sauce.
</p><div class="thumb tleft"><div class="thumbinner" style="width:182px;"><a href="/wiki/File:Alicha_1.jpg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/0/03/Alicha_1.jpg/180px-Alicha_1.jpg" decoding="async" width="180" height="135" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/0/03/Alicha_1.jpg/270px-Alicha_1.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/0/03/Alicha_1.jpg/360px-Alicha_1.jpg 2x" data-file-width="800" data-file-height="600"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Alicha_1.jpg" class="internal" title="Enlarge"></a></div>Typical <a href="/wiki/Ethiopian_cuisine" title="Ethiopian cuisine">Ethiopian</a> and <a href="/wiki/Eritrean_cuisine" title="Eritrean cuisine">Eritrean cuisine</a>: <i><a href="/wiki/Injera" title="Injera">Injera</a></i> (pancake-like bread) and several kinds of <i>wat</i> (stew)</div></div></div><p>The best known Ethio-Eritrean cuisine consists of various vegetable or meat side dishes and entrées, usually a <i><a href="/wiki/Wat_(food)" title="Wat (food)">wat</a></i>, or thick <a href="/wiki/Stew" title="Stew">stew</a>, served atop <i>injera</i>, a large <a href="/wiki/Sourdough" title="Sourdough">sourdough</a> <a href="/wiki/Flatbread" title="Flatbread">flatbread</a> made of teff flour. One does not eat with utensils, but instead uses injera to scoop up the entrées and side dishes.
</p><p>Tihlo, prepared from roasted barley flour, is very popular in Amhara, Agame, and Awlaelo (Tigray). Traditional Ethiopian cuisine employs no <a href="/wiki/Pork" title="Pork">pork</a> or <a href="/wiki/Shellfish" title="Shellfish">shellfish</a> of any kind, as they are forbidden in the Jewish and <a href="/wiki/Ethiopian_Orthodox_Tewahedo_Church" title="Ethiopian Orthodox Tewahedo Church">Ethiopian Orthodox Christian</a> faiths. It is also very common to eat from the same dish in the center of the table with a group of people.
</p><div class="thumb tright"><div class="thumbinner" style="width:222px;"><a href="/wiki/File:Halwo_003.jpg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/d/df/Halwo_003.jpg/220px-Halwo_003.jpg" decoding="async" width="220" height="165" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/d/df/Halwo_003.jpg/330px-Halwo_003.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/d/df/Halwo_003.jpg/440px-Halwo_003.jpg 2x" data-file-width="1632" data-file-height="1224"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Halwo_003.jpg" class="internal" title="Enlarge"></a></div><i>Xalwo</i>, the <a href="/wiki/Somali_people" class="mw-redirect" title="Somali people">Somali</a> version of <a href="/wiki/Halva" title="Halva">halva</a>, is a festive dish in <a href="/wiki/Somali_cuisine" title="Somali cuisine">Somali cuisine</a>.</div></div></div><p><a href="/wiki/Somali_cuisine" title="Somali cuisine">Somali cuisine</a> varies from region to region and consists of an exotic <a href="/wiki/Fusion_cuisine" title="Fusion cuisine">mixture</a> of diverse culinary influences. It is the product of <a href="/wiki/Somalia" title="Somalia">Somalia</a>'s rich <a href="/wiki/Maritime_history_of_Somalia" title="Maritime history of Somalia">tradition of trade and commerce</a>. Despite the variety, there remains one thing that unites the various regional cuisines: all food is served <a href="/wiki/Halal" title="Halal">halal</a>. There are therefore no pork dishes, <a href="/wiki/Alcoholic_beverage" class="mw-redirect" title="Alcoholic beverage">alcohol</a> is not served, nothing that died on its own is eaten, and no blood is incorporated. <i>Qaddo</i> or lunch is often elaborate.
</p><p>Varieties of <i>bariis</i> (rice), the most popular probably being <a href="/wiki/Basmati" title="Basmati">basmati</a>, usually serve as the main dish. Spices like <a href="/wiki/Cumin" title="Cumin">cumin</a>, <a href="/wiki/Cardamom" title="Cardamom">cardamom</a>, <a href="/wiki/Cloves" class="mw-redirect" title="Cloves">cloves</a>, <a href="/wiki/Cinnamon" title="Cinnamon">cinnamon</a> and <a href="/wiki/Common_sage" class="mw-redirect" title="Common sage">sage</a> are used to aromatize these different rice dishes. Somalis serve dinner as late as 9 pm. During <a href="/wiki/Ramadan_(calendar_month)" title="Ramadan (calendar month)">Ramadan</a>, dinner is often served after <a href="/wiki/Tarawih" title="Tarawih">Tarawih</a> prayers, sometimes as late as 11 pm.
</p><p><i>Xalwo</i> (halwo) or <a href="/wiki/Halva" title="Halva">halva</a> is a popular confection served during special occasions such as <a href="/wiki/Eid_ul-Fitr" class="mw-redirect" title="Eid ul-Fitr">Eid</a> celebrations or wedding receptions. It is made from sugar, <a href="/wiki/Cornstarch" class="mw-redirect" title="Cornstarch">cornstarch</a>, <a href="/wiki/Cardamom" title="Cardamom">cardamom</a> powder, <a href="/wiki/Nutmeg" title="Nutmeg">nutmeg</a> powder, and <a href="/wiki/Ghee" title="Ghee">ghee</a>. Peanuts are also sometimes added to enhance texture and flavor.<sup id="cite_ref-17" class="reference"><a href="#cite_note-17">[17]</a></sup> After meals, homes are traditionally perfumed using <a href="/wiki/Frankincense" title="Frankincense">frankincense</a> (<i>lubaan</i>) or <a href="/wiki/Incense" title="Incense">incense</a> (<i>cuunsi</i>), which is prepared inside an <a href="/wiki/Censer" title="Censer">incense burner</a> referred to as a <i><a href="/wiki/Dabqaad" title="Dabqaad">dabqaad</a></i>.
</p>'''

east_africa = ["Djibouti", "Eritrea", "Ethiopia", "Somalia", "Sudan", "South Sudan", "Madagascar", "Mauritius", "Comoros", "Seychelles", "French", "Réunion", "Mayotte", "Uganda", "Rwanda", "Burundi", "Kenya", "Tanzania", "Mozambique", "Malawi", "Zambia", "Zimbabwe"]
east_african_links = get_links_from_raw_html(raw_html, "African_cuisine#East_Africa", "East Africa")

[('African_cuisine#East_Africa', 'East Africa'),
 ('Vicia_faba', 'fava beans'),
 ('Tarawih', 'Tarawih'),
 ('Eritrean_cuisine', 'Eritrean cuisine'),
 ('Incense', 'incense'),
 ('Pigs', 'pigs'),
 ('Sheep', 'sheep'),
 ('Persian_people', 'Persian'),
 ('Yemen', 'Yemeni'),
 ('Curries', 'curries'),
 ('Fufu', 'fufu'),
 ('Sourdough', 'sourdough'),
 ('Ugali', 'Ugali'),
 ('Lentil', 'lentils'),
 ('Pork', 'pork'),
 ('Cattle', 'Cattle'),
 ('Banana', 'bananas'),
 ('East_Africa', 'East Africans'),
 ('Flatbread', 'flatbread'),
 ('Cardamom', 'cardamom'),
 ('Sorghum', 'sorghum'),
 ('Akelet', 'akelet'),
 ('Wat_(food)', 'tsebhis'),
 ('Stew', 'stew'),
 ('Lentil', 'lentil'),
 ('Clove', 'cloves'),
 ('Somalia', 'Somalia'),
 ('Wheat', 'wheat'),
 ('Ramadan_(calendar_month)', 'Ramadan'),
 ('Swahili_culture', 'Swahili'),
 ('Basmati', 'basmati'),
 ('Maize', 'Maize'),
 ('Matoke', 'matoke'),
 ('Cornstarch', 'cornstarch'),
 ('Common_sage', 'sage'),
 ('Oman', 'Omani'),
 ('Censer', 'incense burner'),
 ('Somali_people', '

In [100]:
north_africa = ["Algeria", "Morocco", "Tunisia", "Libya", "Mauritania"]
east_african_links = get_cuisine_links("https://en.wikipedia.org/wiki/Maghrebi_cuisine", "North African cuisine")
get_cuisine_links("https://en.wikipedia.org/wiki/Maghrebi_cuisine", "North African cuisine")


200
200


[('North African cuisine', 'North African cuisine'),
 ('Maghrebi_cuisine', 'Maghrebi cuisine'),
 ('Ras_el_hanout', 'ras el hanout'),
 ('Allspice', 'allspice'),
 ('Category:African_cuisine', 'African cuisine'),
 ('Coriander', 'coriander'),
 ('Date_(fruit)', 'dates'),
 ('Baharat', 'baharat'),
 ('Oxford_University_Press', 'Oxford University Press'),
 ('Category:Arab_cuisine', 'Arab cuisine'),
 ('Maghreb', 'Maghreb'),
 ('Couscous', 'Couscous'),
 ('Fish', 'fish'),
 ('Cayenne_pepper', 'cayenne pepper'),
 ('Spices', 'spices'),
 ('Vegetable', 'vegetables'),
 ('The_Oxford_Companion_to_Food', 'The Oxford Companion to Food'),
 ('Stew', 'stew'),
 ('List_of_African_cuisines', 'List of African cuisines'),
 ('Wheat', 'wheat'),
 ('Pastilla', 'pastilla'),
 ('Libya', 'Libya'),
 ('Staple_foods', 'staple foods'),
 ('Paprika', 'paprika'),
 ('Turmeric', 'turmeric'),
 ('Parsley', 'parsley'),
 ('Caraway', 'caraway'),
 ('Algerian_cuisine', 'Algeria'),
 ('Ginger', 'ginger'),
 ('Tunisian_cuisine', 'Tunisia'),
 (

In [102]:
west_africa = ["Benin", "Burkina Faso", "Cape Verde", "Ivory Coast", "Gambia", "Ghana", "Guinea", "Guinea-Bissau", "Liberia", "Mali", "Mauritania", "Niger", "Nigeria", "Saint Helena", "Ascension", "Tristan da Cunha", "Senegal", "Sierra Leone", "Togo"]
west_african_links = get_cuisine_links("https://en.wikipedia.org/wiki/West_African_cuisine", "West African cuisine")
get_cuisine_links("https://en.wikipedia.org/wiki/West_African_cuisine", "West Africa")


200
200


[('West Africa', 'West Africa'),
 ('West_African_cuisine', 'West African cuisine'),
 ('The_Sahel', 'the Sahel'),
 ('Cuisine_of_Burkina_Faso', 'Cuisine of Burkina Faso'),
 ('Mauritanian_cuisine', 'Mauritanian cuisine'),
 ('Category:African_cuisine', 'African cuisine'),
 ('Parkia_biglobosa', 'Parkia biglobosa'),
 ('Holy_trinity_(cuisine)', 'holy trinity'),
 ('Peanut', 'peanut'),
 ('Cameroon', 'Cameroon'),
 ('Meat_pie', 'Meat pie'),
 ('Palaver_sauce', 'Palaver sauce'),
 ('Doi_(identifier)', 'doi'),
 ('Grains_of_paradise', 'grains of paradise'),
 ('Ogbono_soup', 'Ogbono soup'),
 ('Cornmeal', 'cornmeal'),
 ('Moin_moin', 'Moin moin'),
 ('Ivorian_cuisine', 'Ivorian cuisine'),
 ('Cuisine_of_Niger', 'Cuisine of Niger'),
 ('The_Gambia', 'the Gambia'),
 ('Kelewele', 'kelewele'),
 ('Green_leafy_vegetable', 'green leafy vegetables'),
 ('Fufu', 'Fufu'),
 ('Category:Burkinab%C3%A9_cuisine', 'Burkinabé cuisine'),
 ('Cola_nut', 'Cola nuts'),
 ('Chin_chin', 'Chin chin'),
 ('Cote_d%27Ivoire', "Cote d'Ivo

In [103]:
raw_html = '''<div class="thumb tright"><div class="thumbinner" style="width:202px;"><a href="/wiki/File:Ugali_and_cabbage.jpg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/f/fe/Ugali_and_cabbage.jpg/200px-Ugali_and_cabbage.jpg" decoding="async" width="200" height="175" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/f/fe/Ugali_and_cabbage.jpg/300px-Ugali_and_cabbage.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/f/fe/Ugali_and_cabbage.jpg/400px-Ugali_and_cabbage.jpg 2x" data-file-width="2183" data-file-height="1912"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Ugali_and_cabbage.jpg" class="internal" title="Enlarge"></a></div><i><a href="/wiki/Ugali" title="Ugali">Ugali</a></i> is pictured here with a side dish of <a href="/wiki/Cabbage" title="Cabbage">cabbage</a>, though it is more typically eaten with <a href="/wiki/Collard_greens" class="mw-redirect" title="Collard greens">collard greens</a> (<i>sukuma wiki</i>).</div></div></div><div class="thumb tright"><div class="thumbinner" style="width:222px;"><a href="/wiki/File:Forodhani_park_food_stand.jpg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Forodhani_park_food_stand.jpg/220px-Forodhani_park_food_stand.jpg" decoding="async" width="220" height="147" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Forodhani_park_food_stand.jpg/330px-Forodhani_park_food_stand.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Forodhani_park_food_stand.jpg/440px-Forodhani_park_food_stand.jpg 2x" data-file-width="1600" data-file-height="1067"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Forodhani_park_food_stand.jpg" class="internal" title="Enlarge"></a></div>Barbecued beef cubes and seafood in <a href="/wiki/Forodhani_Gardens" title="Forodhani Gardens">Forodhani Gardens</a>, <a href="/wiki/Zanzibar" title="Zanzibar">Zanzibar</a></div></div></div><div class="thumb tright"><div class="thumbinner" style="width:222px;"><a href="/wiki/File:Kitcha_fit_fit.png" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/b/b5/Kitcha_fit_fit.png/220px-Kitcha_fit_fit.png" decoding="async" width="220" height="231" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/b/b5/Kitcha_fit_fit.png/330px-Kitcha_fit_fit.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/b/b5/Kitcha_fit_fit.png/440px-Kitcha_fit_fit.png 2x" data-file-width="828" data-file-height="868"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Kitcha_fit_fit.png" class="internal" title="Enlarge"></a></div><i><a href="/wiki/Kitcha_fit-fit" class="mw-redirect" title="Kitcha fit-fit">Kitcha fit-fit</a></i>, a mainstay in <a href="/wiki/Eritrean_cuisine" title="Eritrean cuisine">Eritrean cuisine</a>, is presented with a scoop of fresh yoghurt and topped with berbere (spice).</div></div></div><p>The cuisine of <a href="/wiki/East_Africa" title="East Africa">East Africa</a> varies from area to area. In the inland savannah, the traditional cuisine of cattle-keeping peoples is distinctive in that meat products are generally absent. <a href="/wiki/Cattle" title="Cattle">Cattle</a>, <a href="/wiki/Sheep" title="Sheep">sheep</a>, <a href="/wiki/Pigs" class="mw-redirect" title="Pigs">pigs</a> and <a href="/wiki/Goat" title="Goat">goats</a> were regarded as a form of <a href="/wiki/Currency" title="Currency">currency</a><sup id="cite_ref-14" class="reference"><a href="#cite_note-14">[14]</a></sup> and a store of wealth, and are not generally consumed as food.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (October 2017)">citation needed</span></a></i>]</sup>
</p><p>In some areas, traditional <a href="/wiki/East_Africa" title="East Africa">East Africans</a> consume the milk and blood of cattle, but rarely the meat. Elsewhere, other peoples are farmers who grow a variety of grains and vegetables. <a href="/wiki/Maize" title="Maize">Maize</a> (corn) is the basis of <a href="/wiki/Ugali" title="Ugali">ugali</a>, the local version of West and Central Africa's <a href="/wiki/Fufu" title="Fufu">fufu</a>. Ugali is a starch dish eaten with meats or stews. In <a href="/wiki/Uganda" title="Uganda">Uganda</a>, steamed green <a href="/wiki/Banana" title="Banana">bananas</a> called <a href="/wiki/Matoke" class="mw-redirect" title="Matoke">matoke</a> provide the starch filler of many meals.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (October 2017)">citation needed</span></a></i>]</sup>
</p><p>Around 1000&nbsp;years ago, <a href="/wiki/Oman" title="Oman">Omani</a> and <a href="/wiki/Yemen" title="Yemen">Yemeni</a> merchants settled on the <a href="/wiki/Swahili_Coast" class="mw-redirect" title="Swahili Coast">Swahili Coast</a>. Middle Eastern influences are especially reflected in the <a href="/wiki/Swahili_culture" title="Swahili culture">Swahili</a> cuisine of the coast – steamed or cooked rice with spices in <a href="/wiki/Persian_people" class="mw-redirect" title="Persian people">Persian</a> style; <a href="/wiki/Saffron" title="Saffron">saffron</a>, <a href="/wiki/Clove" title="Clove">cloves</a>, <a href="/wiki/Cinnamon" title="Cinnamon">cinnamon</a> and several other spices; and <a href="/wiki/Pomegranate" title="Pomegranate">pomegranate</a> juice.<sup id="cite_ref-15" class="reference"><a href="#cite_note-15">[15]</a></sup>
</p><p>Several centuries later, the British and the Indians came, and both brought with them foods such as Indian spiced vegetable <a href="/wiki/Curries" class="mw-redirect" title="Curries">curries</a>, <a href="/wiki/Lentil" title="Lentil">lentil</a> <a href="/wiki/Soup" title="Soup">soups</a>, chapattis and a variety of <a href="/wiki/Pickling" title="Pickling">pickles</a> which have influenced various local dishes. Some common ingredients used in this region include oranges, lemons, limes, chillis, capsicum peppers, maize, tomatoes, and strawberries. 
</p><p>In the Horn of Africa, the main traditional dishes in <a href="/wiki/Eritrean_cuisine" title="Eritrean cuisine">Eritrean cuisine</a> and <a href="/wiki/Ethiopian_cuisine" title="Ethiopian cuisine">Ethiopian cuisine</a> are <i><a href="/wiki/Wat_(food)" title="Wat (food)">tsebhis</a></i> (stews) served with <i><a href="/wiki/Injera" title="Injera">injera</a></i><sup id="cite_ref-Neo-Africanism_2008_2-1" class="reference"><a href="#cite_note-Neo-Africanism_2008-2">[2]</a></sup><sup id="cite_ref-webcitation_16-0" class="reference"><a href="#cite_note-webcitation-16">[16]</a></sup> (flatbread made from <a href="/wiki/Teff" title="Teff">teff</a>,<sup id="cite_ref-webcitation_16-1" class="reference"><a href="#cite_note-webcitation-16">[16]</a></sup> <a href="/wiki/Wheat" title="Wheat">wheat</a>, or <a href="/wiki/Sorghum" title="Sorghum">sorghum</a>) and <a href="/w/index.php?title=Hilbet&amp;action=edit&amp;redlink=1" class="new" title="Hilbet (page does not exist)">hilbet</a> (paste made from <a href="/wiki/Legume" title="Legume">legumes</a>, mainly <a href="/wiki/Lentil" title="Lentil">lentils</a> and <a href="/wiki/Vicia_faba" title="Vicia faba">fava beans</a>). Eritrean and Ethiopian cuisine (especially in the northern half) are very similar, given the shared history of the two countries.
</p><p>Eritrean and Ethiopian food habits vary regionally. In the <a href="/wiki/Ethiopian_Highlands" title="Ethiopian Highlands">highlands</a>, injera is the staple diet and is eaten daily among the <a href="/wiki/Tigray-Tigrinya_people" class="mw-redirect" title="Tigray-Tigrinya people">Tigrinya</a>. Injera is made out of teff, wheat, barley, sorghum or corn, and resembles a spongy, slightly sour pancake. When eating, diners generally share food from a large tray placed in the center of a low dining table. Numerous injera is layered on this tray and topped with various spicy stews. Diners then break into the section of injera in front of them, tearing off pieces and dipping them into the stews.
</p><p>In the lowlands, the main dish is <i><a href="/wiki/Akelet" class="mw-redirect" title="Akelet">akelet</a></i>, a porridge-like dish made from wheat flour dough. A <a href="/wiki/Ladle_(spoon)" title="Ladle (spoon)">ladle</a> is used to scoop out the top, which is filled with berbere and butter sauce and surrounded by milk or yoghurt. A small piece of dough is broken and then used to scoop up the sauce.
</p><div class="thumb tleft"><div class="thumbinner" style="width:182px;"><a href="/wiki/File:Alicha_1.jpg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/0/03/Alicha_1.jpg/180px-Alicha_1.jpg" decoding="async" width="180" height="135" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/0/03/Alicha_1.jpg/270px-Alicha_1.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/0/03/Alicha_1.jpg/360px-Alicha_1.jpg 2x" data-file-width="800" data-file-height="600"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Alicha_1.jpg" class="internal" title="Enlarge"></a></div>Typical <a href="/wiki/Ethiopian_cuisine" title="Ethiopian cuisine">Ethiopian</a> and <a href="/wiki/Eritrean_cuisine" title="Eritrean cuisine">Eritrean cuisine</a>: <i><a href="/wiki/Injera" title="Injera">Injera</a></i> (pancake-like bread) and several kinds of <i>wat</i> (stew)</div></div></div><p>The best known Ethio-Eritrean cuisine consists of various vegetable or meat side dishes and entrées, usually a <i><a href="/wiki/Wat_(food)" title="Wat (food)">wat</a></i>, or thick <a href="/wiki/Stew" title="Stew">stew</a>, served atop <i>injera</i>, a large <a href="/wiki/Sourdough" title="Sourdough">sourdough</a> <a href="/wiki/Flatbread" title="Flatbread">flatbread</a> made of teff flour. One does not eat with utensils, but instead uses injera to scoop up the entrées and side dishes.
</p><p>Tihlo, prepared from roasted barley flour, is very popular in Amhara, Agame, and Awlaelo (Tigray). Traditional Ethiopian cuisine employs no <a href="/wiki/Pork" title="Pork">pork</a> or <a href="/wiki/Shellfish" title="Shellfish">shellfish</a> of any kind, as they are forbidden in the Jewish and <a href="/wiki/Ethiopian_Orthodox_Tewahedo_Church" title="Ethiopian Orthodox Tewahedo Church">Ethiopian Orthodox Christian</a> faiths. It is also very common to eat from the same dish in the center of the table with a group of people.
</p><div class="thumb tright"><div class="thumbinner" style="width:222px;"><a href="/wiki/File:Halwo_003.jpg" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/d/df/Halwo_003.jpg/220px-Halwo_003.jpg" decoding="async" width="220" height="165" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/d/df/Halwo_003.jpg/330px-Halwo_003.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/d/df/Halwo_003.jpg/440px-Halwo_003.jpg 2x" data-file-width="1632" data-file-height="1224"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Halwo_003.jpg" class="internal" title="Enlarge"></a></div><i>Xalwo</i>, the <a href="/wiki/Somali_people" class="mw-redirect" title="Somali people">Somali</a> version of <a href="/wiki/Halva" title="Halva">halva</a>, is a festive dish in <a href="/wiki/Somali_cuisine" title="Somali cuisine">Somali cuisine</a>.</div></div></div><p><a href="/wiki/Somali_cuisine" title="Somali cuisine">Somali cuisine</a> varies from region to region and consists of an exotic <a href="/wiki/Fusion_cuisine" title="Fusion cuisine">mixture</a> of diverse culinary influences. It is the product of <a href="/wiki/Somalia" title="Somalia">Somalia</a>'s rich <a href="/wiki/Maritime_history_of_Somalia" title="Maritime history of Somalia">tradition of trade and commerce</a>. Despite the variety, there remains one thing that unites the various regional cuisines: all food is served <a href="/wiki/Halal" title="Halal">halal</a>. There are therefore no pork dishes, <a href="/wiki/Alcoholic_beverage" class="mw-redirect" title="Alcoholic beverage">alcohol</a> is not served, nothing that died on its own is eaten, and no blood is incorporated. <i>Qaddo</i> or lunch is often elaborate.
</p><p>Varieties of <i>bariis</i> (rice), the most popular probably being <a href="/wiki/Basmati" title="Basmati">basmati</a>, usually serve as the main dish. Spices like <a href="/wiki/Cumin" title="Cumin">cumin</a>, <a href="/wiki/Cardamom" title="Cardamom">cardamom</a>, <a href="/wiki/Cloves" class="mw-redirect" title="Cloves">cloves</a>, <a href="/wiki/Cinnamon" title="Cinnamon">cinnamon</a> and <a href="/wiki/Common_sage" class="mw-redirect" title="Common sage">sage</a> are used to aromatize these different rice dishes. Somalis serve dinner as late as 9 pm. During <a href="/wiki/Ramadan_(calendar_month)" title="Ramadan (calendar month)">Ramadan</a>, dinner is often served after <a href="/wiki/Tarawih" title="Tarawih">Tarawih</a> prayers, sometimes as late as 11 pm.
</p><p><i>Xalwo</i> (halwo) or <a href="/wiki/Halva" title="Halva">halva</a> is a popular confection served during special occasions such as <a href="/wiki/Eid_ul-Fitr" class="mw-redirect" title="Eid ul-Fitr">Eid</a> celebrations or wedding receptions. It is made from sugar, <a href="/wiki/Cornstarch" class="mw-redirect" title="Cornstarch">cornstarch</a>, <a href="/wiki/Cardamom" title="Cardamom">cardamom</a> powder, <a href="/wiki/Nutmeg" title="Nutmeg">nutmeg</a> powder, and <a href="/wiki/Ghee" title="Ghee">ghee</a>. Peanuts are also sometimes added to enhance texture and flavor.<sup id="cite_ref-17" class="reference"><a href="#cite_note-17">[17]</a></sup> After meals, homes are traditionally perfumed using <a href="/wiki/Frankincense" title="Frankincense">frankincense</a> (<i>lubaan</i>) or <a href="/wiki/Incense" title="Incense">incense</a> (<i>cuunsi</i>), which is prepared inside an <a href="/wiki/Censer" title="Censer">incense burner</a> referred to as a <i><a href="/wiki/Dabqaad" title="Dabqaad">dabqaad</a></i>.
</p>'''

southern_africa = ["Botswana", "Eswatini", "Lesotho", "Namibia", "South Africa"]
southern_african_links = get_links_from_raw_html(raw_html, "African_cuisine#Southern_Africa", "Southern Africa")
get_links_from_raw_html(raw_html, "African_cuisine#Southern_Africa", "Southern Africa")

[('African_cuisine#Southern_Africa', 'Southern Africa'),
 ('Vicia_faba', 'fava beans'),
 ('Tarawih', 'Tarawih'),
 ('Eritrean_cuisine', 'Eritrean cuisine'),
 ('Incense', 'incense'),
 ('Pigs', 'pigs'),
 ('Sheep', 'sheep'),
 ('Persian_people', 'Persian'),
 ('Yemen', 'Yemeni'),
 ('Curries', 'curries'),
 ('Fufu', 'fufu'),
 ('Sourdough', 'sourdough'),
 ('Ugali', 'Ugali'),
 ('Lentil', 'lentils'),
 ('Pork', 'pork'),
 ('Cattle', 'Cattle'),
 ('Banana', 'bananas'),
 ('East_Africa', 'East Africans'),
 ('Flatbread', 'flatbread'),
 ('Cardamom', 'cardamom'),
 ('Sorghum', 'sorghum'),
 ('Akelet', 'akelet'),
 ('Wat_(food)', 'tsebhis'),
 ('Stew', 'stew'),
 ('Lentil', 'lentil'),
 ('Clove', 'cloves'),
 ('Somalia', 'Somalia'),
 ('Wheat', 'wheat'),
 ('Ramadan_(calendar_month)', 'Ramadan'),
 ('Swahili_culture', 'Swahili'),
 ('Basmati', 'basmati'),
 ('Maize', 'Maize'),
 ('Matoke', 'matoke'),
 ('Cornstarch', 'cornstarch'),
 ('Common_sage', 'sage'),
 ('Oman', 'Omani'),
 ('Censer', 'incense burner'),
 ('Somali_pe

In [104]:
# Included "Overview" from South American cuisine
raw_html = '''<p>The <a href="/wiki/Amazon_basin" title="Amazon basin">Amazon basin</a> of <a href="/wiki/South_America" title="South America">South America</a> provides a plethora of fresh fish and <a href="/wiki/Tropical_fruit" class="mw-redirect" title="Tropical fruit">tropical fruits</a>.<sup id="cite_ref-3" class="reference"><a href="#cite_note-3">[3]</a></sup> The <a href="/wiki/Pacific_Ocean" title="Pacific Ocean">Pacific Ocean</a> provides a large amount of <a href="/wiki/Seafood" title="Seafood">seafood</a>, such as <a href="/wiki/King_crab" title="King crab">king crab</a> (typically caught at the southern end of the continent), <a href="/wiki/Lobster" title="Lobster">lobster</a> (found in great quantities from the <a href="/wiki/Juan_Fern%C3%A1ndez_Islands" title="Juan Fernández Islands">Juan Fernández Islands</a>), and <a href="/wiki/Antarctic_krill" title="Antarctic krill">Antarctic krill</a>, which was recently discovered. <a href="/wiki/Tuna" title="Tuna">Tuna</a> and tropical fish are caught all around the continent but are notably found in abundance near <a href="/wiki/Easter_Island" title="Easter Island">Easter Island</a>. The many plains on this continent make it rich for growing foods like <a href="/wiki/Cereals" class="mw-redirect" title="Cereals">cereals</a>, <a href="/wiki/Potatoes" class="mw-redirect" title="Potatoes">potatoes</a> and <a href="/wiki/Quinoa" title="Quinoa">quinoa</a>. In the <a href="/wiki/Patagonia" title="Patagonia">Patagonia</a> region in the south of <a href="/wiki/Argentina" title="Argentina">Argentina</a>, many people raise <a href="/wiki/Domestic_sheep" class="mw-redirect" title="Domestic sheep">sheep</a> and <a href="/wiki/Cattle" title="Cattle">cattle</a>. In <a href="/wiki/Brazil" title="Brazil">Brazil</a>, a traditional dish is the <a href="/wiki/Feijoada" title="Feijoada">feijoada</a>, a <a href="/wiki/Stew" title="Stew">stew</a> consisting of <a href="/wiki/Common_beans" class="mw-redirect" title="Common beans">beans</a> with <a href="/wiki/Beef" title="Beef">beef</a> and <a href="/wiki/Pork" title="Pork">pork</a>.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (July 2011)">citation needed</span></a></i>]</sup> <a href="/wiki/Rice_and_beans" title="Rice and beans">Rice and beans</a> is also very common in Brazil.<sup id="cite_ref-4" class="reference"><a href="#cite_note-4">[4]</a></sup> <a href="/wiki/Peruvian_cuisine" title="Peruvian cuisine">Peruvian cuisine</a> is largely influenced by traditional <a href="/wiki/Incan" class="mw-redirect" title="Incan">Incan</a> culture (see <a href="/wiki/Inca_cuisine" title="Inca cuisine">Incan cuisine</a>). The consumption of <a href="/wiki/Carne_asada" title="Carne asada">carne asada</a> is widespread throughout much of the continent.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (January 2018)">citation needed</span></a></i>]</sup><sup id="cite_ref-7" class="reference"><a href="#cite_note-7">[a]</a></sup>  It is popular to grill in the open air as in <a href="/wiki/Asado" title="Asado">asado</a> or <a href="/wiki/Churrasco" title="Churrasco">churrasco</a>. <a href="/wiki/Shredded_beef" title="Shredded beef">Shredded beef</a> is common throughout South America and Latin America and is served with a variety of foods.<sup id="cite_ref-Gutierrez_2013_p._150_8-0" class="reference"><a href="#cite_note-Gutierrez_2013_p._150-8">[5]</a></sup>
</p><div class="thumb tright"><div class="thumbinner" style="width:222px;"><a href="/wiki/File:Pachamanca_Peru.JPG" class="image"><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/9/9d/Pachamanca_Peru.JPG/220px-Pachamanca_Peru.JPG" decoding="async" width="220" height="165" class="thumbimage" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/9/9d/Pachamanca_Peru.JPG/330px-Pachamanca_Peru.JPG 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/9/9d/Pachamanca_Peru.JPG/440px-Pachamanca_Peru.JPG 2x" data-file-width="2048" data-file-height="1536"></a>  <div class="thumbcaption"><div class="magnify"><a href="/wiki/File:Pachamanca_Peru.JPG" class="internal" title="Enlarge"></a></div><a href="/wiki/Pachamanca" title="Pachamanca">Pachamanca</a></div></div></div><div role="note" class="hatnote navigation-not-searchable">Main articles: <a href="/wiki/Argentine_cuisine" title="Argentine cuisine">Argentine cuisine</a>, <a href="/wiki/Bolivian_cuisine" title="Bolivian cuisine">Bolivian cuisine</a>, <a href="/wiki/Chilean_cuisine" title="Chilean cuisine">Chilean cuisine</a>, <a href="/wiki/Ecuadorian_cuisine" title="Ecuadorian cuisine">Ecuadorian cuisine</a>, and <a href="/wiki/Peruvian_cuisine" title="Peruvian cuisine">Peruvian cuisine</a></div><p>The food of the Andes is highly influenced by the <a href="/wiki/Indigenous_peoples" title="Indigenous peoples">indigenous peoples</a>. The principal foods continue to be corn, potatoes<sup id="cite_ref-Krögel_2010_p._20_9-0" class="reference"><a href="#cite_note-Krögel_2010_p._20-9">[6]</a></sup> and other <a href="/wiki/Tuber" title="Tuber">tubers</a>. The meats most characteristic of this zone are the llama (Peru) and the guinea pig (Chile, Bolivia, Peru, Colombia and the Argentine northwest). In areas where there is fresh water, trout is consumed. <i><a href="/wiki/Chupe_andino" title="Chupe andino">Chupe andino</a></i> refers to various <a href="/wiki/Stew" title="Stew">stews</a><sup id="cite_ref-10" class="reference"><a href="#cite_note-10">[7]</a></sup> and <a href="/wiki/Soup" title="Soup">soups</a> that are prepared in the Andes mountains region. One of the most important drinks is <a href="/wiki/Chicha" title="Chicha">chicha</a>.<sup id="cite_ref-11" class="reference"><a href="#cite_note-11">[8]</a></sup>  Important dishes include <a href="/wiki/Humitas" class="mw-redirect" title="Humitas">humitas</a>, <a href="/wiki/Locro" title="Locro">locro</a>, chanfaina, <a href="/wiki/Arepas" class="mw-redirect" title="Arepas">arepas</a>, quimbolitos (dessert tamales), and peppers. A famous dish from the Peruvian Andes is <a href="/wiki/Pachamanca" title="Pachamanca">pachamanca</a>.<sup id="cite_ref-12" class="reference"><a href="#cite_note-12">[9]</a></sup> From the mixture of German, native cuisine, and the <a href="/wiki/Chilo%C3%A9_Archipelago" title="Chiloé Archipelago">Chiloé Archipelago</a> in the southern Andes comes <a href="/wiki/Valdiviano" title="Valdiviano">valdiviano</a> and <a href="/wiki/Curanto" title="Curanto">curanto</a>.
</p><p>The wetter areas of Peru produce sugar cane, lemon, bananas, and oranges. <a href="/wiki/Chancaca" title="Chancaca">Chancaca</a> is popular as well as <a href="/wiki/Carbonara" title="Carbonara">carbonara</a>, <a href="/wiki/Sancocho" title="Sancocho">sancocho</a>, huevos quimbos, potato pie, and <a href="/wiki/Ch%27arki" class="mw-redirect" title="Ch'arki">ch'arki</a>.
</p>'''

andes_south_america = ["Argentina", "Bolivia", "Chile", "Ecuador", "Peruvian"]
andes_south_american_links = get_links_from_raw_html(raw_html, "South_American_cuisine#Andes", "Andes")
get_links_from_raw_html(raw_html, "South_American_cuisine#Andes", "Andes")

[('South_American_cuisine#Andes', 'Andes'),
 ('Stew', 'stew'),
 ('Patagonia', 'Patagonia'),
 ('Soup', 'soups'),
 ('Argentina', 'Argentina'),
 ('Asado', 'asado'),
 ('Ch%27arki', "ch'arki"),
 ('Valdiviano', 'valdiviano'),
 ('Stew', 'stews'),
 ('Quinoa', 'quinoa'),
 ('Amazon_basin', 'Amazon basin'),
 ('Seafood', 'seafood'),
 ('Rice_and_beans', 'Rice and beans'),
 ('Peruvian_cuisine', 'Peruvian cuisine'),
 ('Pachamanca', 'Pachamanca'),
 ('Tuber', 'tubers'),
 ('Humitas', 'humitas'),
 ('Carne_asada', 'carne asada'),
 ('Chilo%C3%A9_Archipelago', 'Chiloé Archipelago'),
 ('Chancaca', 'Chancaca'),
 ('Tuna', 'Tuna'),
 ('Argentine_cuisine', 'Argentine cuisine'),
 ('Feijoada', 'feijoada'),
 ('Domestic_sheep', 'sheep'),
 ('Pacific_Ocean', 'Pacific Ocean'),
 ('Antarctic_krill', 'Antarctic krill'),
 ('Lobster', 'lobster'),
 ('King_crab', 'king crab'),
 ('Brazil', 'Brazil'),
 ('Easter_Island', 'Easter Island'),
 ('Cereals', 'cereals'),
 ('Carbonara', 'carbonara'),
 ('Chupe_andino', 'Chupe andino'),
 ('

In [105]:
# Included "Overview" from South American cuisine
raw_html = '''<p>The <a href="/wiki/Amazon_basin" title="Amazon basin">Amazon basin</a> of <a href="/wiki/South_America" title="South America">South America</a> provides a plethora of fresh fish and <a href="/wiki/Tropical_fruit" class="mw-redirect" title="Tropical fruit">tropical fruits</a>.<sup id="cite_ref-3" class="reference"><a href="#cite_note-3">[3]</a></sup> The <a href="/wiki/Pacific_Ocean" title="Pacific Ocean">Pacific Ocean</a> provides a large amount of <a href="/wiki/Seafood" title="Seafood">seafood</a>, such as <a href="/wiki/King_crab" title="King crab">king crab</a> (typically caught at the southern end of the continent), <a href="/wiki/Lobster" title="Lobster">lobster</a> (found in great quantities from the <a href="/wiki/Juan_Fern%C3%A1ndez_Islands" title="Juan Fernández Islands">Juan Fernández Islands</a>), and <a href="/wiki/Antarctic_krill" title="Antarctic krill">Antarctic krill</a>, which was recently discovered. <a href="/wiki/Tuna" title="Tuna">Tuna</a> and tropical fish are caught all around the continent but are notably found in abundance near <a href="/wiki/Easter_Island" title="Easter Island">Easter Island</a>. The many plains on this continent make it rich for growing foods like <a href="/wiki/Cereals" class="mw-redirect" title="Cereals">cereals</a>, <a href="/wiki/Potatoes" class="mw-redirect" title="Potatoes">potatoes</a> and <a href="/wiki/Quinoa" title="Quinoa">quinoa</a>. In the <a href="/wiki/Patagonia" title="Patagonia">Patagonia</a> region in the south of <a href="/wiki/Argentina" title="Argentina">Argentina</a>, many people raise <a href="/wiki/Domestic_sheep" class="mw-redirect" title="Domestic sheep">sheep</a> and <a href="/wiki/Cattle" title="Cattle">cattle</a>. In <a href="/wiki/Brazil" title="Brazil">Brazil</a>, a traditional dish is the <a href="/wiki/Feijoada" title="Feijoada">feijoada</a>, a <a href="/wiki/Stew" title="Stew">stew</a> consisting of <a href="/wiki/Common_beans" class="mw-redirect" title="Common beans">beans</a> with <a href="/wiki/Beef" title="Beef">beef</a> and <a href="/wiki/Pork" title="Pork">pork</a>.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (July 2011)">citation needed</span></a></i>]</sup> <a href="/wiki/Rice_and_beans" title="Rice and beans">Rice and beans</a> is also very common in Brazil.<sup id="cite_ref-4" class="reference"><a href="#cite_note-4">[4]</a></sup> <a href="/wiki/Peruvian_cuisine" title="Peruvian cuisine">Peruvian cuisine</a> is largely influenced by traditional <a href="/wiki/Incan" class="mw-redirect" title="Incan">Incan</a> culture (see <a href="/wiki/Inca_cuisine" title="Inca cuisine">Incan cuisine</a>). The consumption of <a href="/wiki/Carne_asada" title="Carne asada">carne asada</a> is widespread throughout much of the continent.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (January 2018)">citation needed</span></a></i>]</sup><sup id="cite_ref-7" class="reference"><a href="#cite_note-7">[a]</a></sup>  It is popular to grill in the open air as in <a href="/wiki/Asado" title="Asado">asado</a> or <a href="/wiki/Churrasco" title="Churrasco">churrasco</a>. <a href="/wiki/Shredded_beef" title="Shredded beef">Shredded beef</a> is common throughout South America and Latin America and is served with a variety of foods.<sup id="cite_ref-Gutierrez_2013_p._150_8-0" class="reference"><a href="#cite_note-Gutierrez_2013_p._150-8">[5]</a></sup><div role="note" class="hatnote navigation-not-searchable">See also: <a href="/wiki/Brazilian_cuisine" title="Brazilian cuisine">Brazilian cuisine</a>, <a href="/wiki/Colombian_cuisine" title="Colombian cuisine">Colombian cuisine</a>, <a href="/wiki/Paraguayan_cuisine" title="Paraguayan cuisine">Paraguayan cuisine</a>, and <a href="/wiki/Venezuelan_cuisine" title="Venezuelan cuisine">Venezuelan cuisine</a></div><p>The tropical region of the continent is divided into two distinct areas, the coastal areas of the Atlantic and the Pacific and the Amazon area each with its distinct cuisines. Much of the fruits that are considered to be exotic are common in the tropical forests and fields, such as guava, pineapple, papaya, mango, banana, and elderberry.
</p><p>The climate and geography also favour a great variety of crops: potatoes, sweet potatoes, <a href="/wiki/Cassava" title="Cassava">cassava</a>, complemented with meat and fish; grains, principally rice, corn, and wheat and beans.
</p><p>In the coastal area <a href="/wiki/Ceviche" title="Ceviche">ceviche</a>, <a href="/wiki/Tostones" title="Tostones">tostones</a> or patacón, <a href="/wiki/Arepa" title="Arepa">arepa</a>, <a href="/wiki/Chipa" title="Chipa">chipa</a>, <a href="/wiki/Sancocho" title="Sancocho">sancocho</a>, <a href="/wiki/Pabell%C3%B3n_criollo" title="Pabellón criollo">pabellón criollo</a>, <a href="/wiki/Bandeja_paisa" title="Bandeja paisa">bandeja paisa</a>, <a href="/wiki/Guatita" class="mw-redirect" title="Guatita">guatita</a>, and <a href="/wiki/Sopa_paraguaya" title="Sopa paraguaya">sopa paraguaya</a> are common dishes.
</p><p>In Brazilian <a href="/wiki/Feijoada" title="Feijoada">feijoada</a>,<sup id="cite_ref-13" class="reference"><a href="#cite_note-13">[10]</a></sup> arroz carreteiro (cart riders rice), <a href="/wiki/Coxinha" title="Coxinha">coxinha</a>,<sup id="cite_ref-14" class="reference"><a href="#cite_note-14">[11]</a></sup> and <a href="/wiki/Farofa" title="Farofa">farofa</a> are common; <a href="/wiki/Bahia" title="Bahia">Bahia</a>, a state in Brazil, has its own cuisine which has heavy African influences.
</p><p>The Amazon area is known for its utilization of native meats such as the <a href="/wiki/Capybara" title="Capybara">capybara</a>, turtles, <a href="/wiki/Peccary" title="Peccary">peccary</a>,  and <a href="/wiki/Paca" title="Paca">paca</a>. Common dishes are <a href="/wiki/Juane" title="Juane">juane</a>, <a href="/wiki/Tacacho" title="Tacacho">tacacho</a>, <a href="/wiki/Tacac%C3%A1" title="Tacacá">tacacá</a>. There are a wide variety of fruits native to the Amazon with which a great variety of drinks are prepared.
</p>'''

tropical_south_america = ["Brazil", "Colombia", "Paraguay", "Venezuela"]
tropical_south_american_links = get_links_from_raw_html(raw_html, "South_American_cuisine#Tropical", "Tropical")
get_links_from_raw_html(raw_html, "South_American_cuisine#Tropical", "Tropical")

[('South_American_cuisine#Tropical', 'Tropical'),
 ('Stew', 'stew'),
 ('Ceviche', 'ceviche'),
 ('Patagonia', 'Patagonia'),
 ('Chipa', 'chipa'),
 ('Tostones', 'tostones'),
 ('Guatita', 'guatita'),
 ('Argentina', 'Argentina'),
 ('Asado', 'asado'),
 ('Sopa_paraguaya', 'sopa paraguaya'),
 ('Paraguayan_cuisine', 'Paraguayan cuisine'),
 ('Quinoa', 'quinoa'),
 ('Amazon_basin', 'Amazon basin'),
 ('Seafood', 'seafood'),
 ('Rice_and_beans', 'Rice and beans'),
 ('Peruvian_cuisine', 'Peruvian cuisine'),
 ('Bandeja_paisa', 'bandeja paisa'),
 ('Juane', 'juane'),
 ('Carne_asada', 'carne asada'),
 ('Paca', 'paca'),
 ('Tuna', 'Tuna'),
 ('Feijoada', 'feijoada'),
 ('Colombian_cuisine', 'Colombian cuisine'),
 ('Domestic_sheep', 'sheep'),
 ('Brazilian_cuisine', 'Brazilian cuisine'),
 ('Pabell%C3%B3n_criollo', 'pabellón criollo'),
 ('Pacific_Ocean', 'Pacific Ocean'),
 ('Antarctic_krill', 'Antarctic krill'),
 ('Lobster', 'lobster'),
 ('King_crab', 'king crab'),
 ('Brazil', 'Brazil'),
 ('Easter_Island', 'East

In [106]:
# Included "Overview" from South American cuisine
raw_html = '''<p>The <a href="/wiki/Amazon_basin" title="Amazon basin">Amazon basin</a> of <a href="/wiki/South_America" title="South America">South America</a> provides a plethora of fresh fish and <a href="/wiki/Tropical_fruit" class="mw-redirect" title="Tropical fruit">tropical fruits</a>.<sup id="cite_ref-3" class="reference"><a href="#cite_note-3">[3]</a></sup> The <a href="/wiki/Pacific_Ocean" title="Pacific Ocean">Pacific Ocean</a> provides a large amount of <a href="/wiki/Seafood" title="Seafood">seafood</a>, such as <a href="/wiki/King_crab" title="King crab">king crab</a> (typically caught at the southern end of the continent), <a href="/wiki/Lobster" title="Lobster">lobster</a> (found in great quantities from the <a href="/wiki/Juan_Fern%C3%A1ndez_Islands" title="Juan Fernández Islands">Juan Fernández Islands</a>), and <a href="/wiki/Antarctic_krill" title="Antarctic krill">Antarctic krill</a>, which was recently discovered. <a href="/wiki/Tuna" title="Tuna">Tuna</a> and tropical fish are caught all around the continent but are notably found in abundance near <a href="/wiki/Easter_Island" title="Easter Island">Easter Island</a>. The many plains on this continent make it rich for growing foods like <a href="/wiki/Cereals" class="mw-redirect" title="Cereals">cereals</a>, <a href="/wiki/Potatoes" class="mw-redirect" title="Potatoes">potatoes</a> and <a href="/wiki/Quinoa" title="Quinoa">quinoa</a>. In the <a href="/wiki/Patagonia" title="Patagonia">Patagonia</a> region in the south of <a href="/wiki/Argentina" title="Argentina">Argentina</a>, many people raise <a href="/wiki/Domestic_sheep" class="mw-redirect" title="Domestic sheep">sheep</a> and <a href="/wiki/Cattle" title="Cattle">cattle</a>. In <a href="/wiki/Brazil" title="Brazil">Brazil</a>, a traditional dish is the <a href="/wiki/Feijoada" title="Feijoada">feijoada</a>, a <a href="/wiki/Stew" title="Stew">stew</a> consisting of <a href="/wiki/Common_beans" class="mw-redirect" title="Common beans">beans</a> with <a href="/wiki/Beef" title="Beef">beef</a> and <a href="/wiki/Pork" title="Pork">pork</a>.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (July 2011)">citation needed</span></a></i>]</sup> <a href="/wiki/Rice_and_beans" title="Rice and beans">Rice and beans</a> is also very common in Brazil.<sup id="cite_ref-4" class="reference"><a href="#cite_note-4">[4]</a></sup> <a href="/wiki/Peruvian_cuisine" title="Peruvian cuisine">Peruvian cuisine</a> is largely influenced by traditional <a href="/wiki/Incan" class="mw-redirect" title="Incan">Incan</a> culture (see <a href="/wiki/Inca_cuisine" title="Inca cuisine">Incan cuisine</a>). The consumption of <a href="/wiki/Carne_asada" title="Carne asada">carne asada</a> is widespread throughout much of the continent.<sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (January 2018)">citation needed</span></a></i>]</sup><sup id="cite_ref-7" class="reference"><a href="#cite_note-7">[a]</a></sup>  It is popular to grill in the open air as in <a href="/wiki/Asado" title="Asado">asado</a> or <a href="/wiki/Churrasco" title="Churrasco">churrasco</a>. <a href="/wiki/Shredded_beef" title="Shredded beef">Shredded beef</a> is common throughout South America and Latin America and is served with a variety of foods.<sup id="cite_ref-Gutierrez_2013_p._150_8-0" class="reference"><a href="#cite_note-Gutierrez_2013_p._150-8">[5]</a></sup>
</p><div role="note" class="hatnote navigation-not-searchable">Main articles: <a href="/wiki/Argentine_cuisine" title="Argentine cuisine">Argentine cuisine</a> and <a href="/wiki/Uruguayan_cuisine" title="Uruguayan cuisine">Uruguayan cuisine</a></div><p>The <a href="/wiki/Pampas" title="Pampas">pampas</a> have the most Italian and German influences. In Argentina, they are the center of the three typical Argentinian dishes <a href="/wiki/Dulce_de_leche" title="Dulce de leche">dulce de leche</a>, <a href="/wiki/Asado" title="Asado">asado</a> (churrasco in Brazil), and <a href="/wiki/Milanesa" title="Milanesa">milanesa</a>. Argentina pizza is different from Italian pizza, being closer to <a href="/wiki/Calzone" title="Calzone">calzones</a>. Pasta and polenta are common in Argentina and on the Pampas generally. <a href="/wiki/Empanadas" class="mw-redirect" title="Empanadas">Empanadas</a> and <a href="/wiki/Chorip%C3%A1n" title="Choripán">choripán</a> are very popular fast food in Argentina and Uruguay. <a href="/wiki/Churros" class="mw-redirect" title="Churros">Churros</a>, <a href="/wiki/Ensa%C3%AFmada" title="Ensaïmada">ensaïmada</a>, <a href="/wiki/Alfajor" title="Alfajor">alfajor</a>, Spanish tortillas with potato, meatballs, <a href="/wiki/Sopa_de_mondongo" title="Sopa de mondongo">sopa de mondongo</a>, and <a href="/wiki/Puchero" title="Puchero">puchero</a> are Spanish derived Pampas cuisine. <a href="/wiki/Mate_(drink)" title="Mate (drink)">Mate</a> is also found on the Pampas.<sup id="cite_ref-Henty_2016_15-0" class="reference"><a href="#cite_note-Henty_2016-15">[12]</a></sup>
</p>'''

pampas_south_america = ["Argentina", "Uruguay"]
pampas_south_american_links = get_links_from_raw_html(raw_html, "South_American_cuisine#Pampas", "Pampas")
get_links_from_raw_html(raw_html, "South_American_cuisine#Pampas", "Pampas")

[('South_American_cuisine#Pampas', 'Pampas'),
 ('Stew', 'stew'),
 ('Ensa%C3%AFmada', 'ensaïmada'),
 ('Patagonia', 'Patagonia'),
 ('Sopa_de_mondongo', 'sopa de mondongo'),
 ('Argentina', 'Argentina'),
 ('Asado', 'asado'),
 ('Dulce_de_leche', 'dulce de leche'),
 ('Quinoa', 'quinoa'),
 ('Amazon_basin', 'Amazon basin'),
 ('Seafood', 'seafood'),
 ('Rice_and_beans', 'Rice and beans'),
 ('Peruvian_cuisine', 'Peruvian cuisine'),
 ('Carne_asada', 'carne asada'),
 ('Alfajor', 'alfajor'),
 ('Tuna', 'Tuna'),
 ('Argentine_cuisine', 'Argentine cuisine'),
 ('Feijoada', 'feijoada'),
 ('Uruguayan_cuisine', 'Uruguayan cuisine'),
 ('Domestic_sheep', 'sheep'),
 ('Pacific_Ocean', 'Pacific Ocean'),
 ('Antarctic_krill', 'Antarctic krill'),
 ('Lobster', 'lobster'),
 ('King_crab', 'king crab'),
 ('Brazil', 'Brazil'),
 ('Easter_Island', 'Easter Island'),
 ('Cereals', 'cereals'),
 ('Pampas', 'pampas'),
 ('Churros', 'Churros'),
 ('Potatoes', 'potatoes'),
 ('Incan', 'Incan'),
 ('Inca_cuisine', 'Incan cuisine'),
 (

In [108]:
raw_html = '''<p><b>Latin American cuisine</b> is the typical foods, beverages, and cooking styles common to many of the countries and cultures in <a href="/wiki/Latin_America" title="Latin America">Latin America</a>. Latin America is a highly diverse area of land whose nations have varying cuisines. Some items typical of Latin American cuisine include <a href="/wiki/Maize" title="Maize">maize</a>-based dishes <a href="/wiki/Arepa" title="Arepa">arepas</a>, <a href="/wiki/Pupusa" title="Pupusa">pupusas</a>, <a href="/wiki/Taco" title="Taco">tacos</a>, <a href="/wiki/Tamale" title="Tamale">tamales</a>, <a href="/wiki/Tortilla" title="Tortilla">tortillas</a> and various <a href="/wiki/Salsa_(sauce)" title="Salsa (sauce)">salsas</a> and other condiments (<a href="/wiki/Guacamole" title="Guacamole">guacamole</a>, <a href="/wiki/Pico_de_gallo" title="Pico de gallo">pico de gallo</a>, <a href="/wiki/Mole_(sauce)" class="mw-redirect" title="Mole (sauce)">mole</a>, <a href="/wiki/Chimichurri" title="Chimichurri">chimichurri</a>, <a href="/wiki/Chili_pepper" title="Chili pepper">chili</a>, <a href="/wiki/Aj%C3%AD_(sauce)" title="Ají (sauce)">aji</a>, <a href="/wiki/Pebre" title="Pebre">pebre</a>). <a href="/wiki/Sofrito" title="Sofrito">Sofrito</a>, a culinary term that originally referred to a specific combination of sautéed or braised aromatics, exists in Latin American cuisine. It refers to a sauce of tomatoes, roasted bell peppers, garlic, onions and herbs.
</p><p>Latin American beverages are just as distinct as their foods. Some of the beverages can even date back to the times of the Native Americans. Some popular beverages include <a href="/wiki/Caf%C3%A9" class="mw-redirect" title="Café">coffee</a>, <a href="/wiki/Mate_(beverage)" class="mw-redirect" title="Mate (beverage)">mate</a>, <a href="/wiki/Hibiscus_tea" title="Hibiscus tea">hibiscus tea</a>, <a href="/wiki/Horchata" title="Horchata">horchata</a>, <a href="/wiki/Chicha" title="Chicha">chicha</a>, <a href="/wiki/Atole" title="Atole">atole</a>, <a href="/wiki/Hot_chocolate" title="Hot chocolate">cacao</a> and <a href="/wiki/Agua_fresca" class="mw-redirect" title="Agua fresca">aguas frescas</a>.
</p><p>Desserts in Latin America are generally very sweet in taste. They include <a href="/wiki/Dulce_de_leche" title="Dulce de leche">dulce de leche</a>, <a href="/wiki/Alfajor" title="Alfajor">alfajor</a>, <a href="/wiki/Rice_pudding" title="Rice pudding">rice pudding</a>, <a href="/wiki/Tres_leches_cake" title="Tres leches cake">tres leches cake</a>, <a href="/wiki/Teja_(confectionery)" title="Teja (confectionery)">teja</a> and <a href="/wiki/Cr%C3%A8me_caramel" title="Crème caramel">flan</a>.
</p>'''

latin_america = ["Paraguay", "Mexico", "Colombia", "Ecuador", "Bolivia", "Peru", "Venezuela", "Argentina", "Panama", "Chile", "Costa Rica", "Brazil", "Dominican Republic", "Puerto Rico", "El Salvador", "Guatemala", "Nicaragua", "Honduras", "Uruguay"]
latin_american_links = get_links_from_raw_html(raw_html, "Latin_American_cuisine", "Latin America")
get_links_from_raw_html(raw_html, "Latin_American_cuisine", "Latin America")

[('Latin_American_cuisine', 'Latin America'),
 ('Chili_pepper', 'chili'),
 ('Rice_pudding', 'rice pudding'),
 ('Cr%C3%A8me_caramel', 'flan'),
 ('Guacamole', 'guacamole'),
 ('Tres_leches_cake', 'tres leches cake'),
 ('Dulce_de_leche', 'dulce de leche'),
 ('Pupusa', 'pupusas'),
 ('Chimichurri', 'chimichurri'),
 ('Mate_(beverage)', 'mate'),
 ('Alfajor', 'alfajor'),
 ('Teja_(confectionery)', 'teja'),
 ('Tortilla', 'tortillas'),
 ('Agua_fresca', 'aguas frescas'),
 ('Atole', 'atole'),
 ('Hot_chocolate', 'cacao'),
 ('Caf%C3%A9', 'coffee'),
 ('Maize', 'maize'),
 ('Arepa', 'arepas'),
 ('Chicha', 'chicha'),
 ('Pebre', 'pebre'),
 ('Aj%C3%AD_(sauce)', 'aji'),
 ('Salsa_(sauce)', 'salsas'),
 ('Sofrito', 'Sofrito'),
 ('Taco', 'tacos'),
 ('Latin_America', 'Latin America'),
 ('Tamale', 'tamales'),
 ('Mole_(sauce)', 'mole'),
 ('Pico_de_gallo', 'pico de gallo'),
 ('Hibiscus_tea', 'hibiscus tea'),
 ('Horchata', 'horchata')]